In [93]:
!pip install torch torchvision matplotlib numpy
!pip install wandb

### Import necessary Libraries

In [94]:
import json
import torch
import torch.nn as nn
import numpy as np
import wandb
from tqdm import tqdm
import pandas as pd
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import accuracy_score
import os
import gc

In [95]:
# Set environment variable for max_split_size_mb
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

In [96]:
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

# Get the available devices
devices = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
print('number of available GPU: ',torch.cuda.device_count())  # Should print the number of visible devices

# Print the available devices
print("Available devices:")
for device in devices:
    print(f"- {device}")
device = torch.device("cuda:0")  # Use the first GPU

# Print the current device
#print(f"Current device: {torch.cuda.current_device()}")

number of available GPU:  1
Available devices:
- <torch.cuda.device object at 0x7fe55703e450>


In [97]:
torch.cuda.mem_get_info(device=device)

(6859390976, 8512602112)

In [98]:
!nvidia-smi

Mon Sep  9 16:24:01 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P4000        Off  | 00000000:65:00.0 Off |                  N/A |
| 46%   39C    P8     5W / 105W |   1576MiB /  8118MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [99]:
# Multiclass classification
#Predict if an asset will fail within two different intervals related to the two different decisions


`os`: This module provides functions for interacting with the operating system. It's commonly used for tasks such as file manipulation and directory operations.<br>
`sklearn.preprocessing`: This module from scikit-learn provides functions for preprocessing data, such as scaling, normalization, and encoding categorical variables.<br>
`sklearn.metrics`: This module contains functions for evaluating model performance, such as computing confusion matrices, recall scores, and precision scores.<br>
`multiclass_model_w1_30.h5`:The .h5 extension indicates that the model will be saved in the Hierarchical Data Format version 5 (HDF5) format, which is commonly used for storing large numerical datasets. The model will be saved with the filename **multiclass_model_w1_30.h5.**

In [100]:
# Setting seed for reproducibility
np.random.seed(1234)
PYTHONHASHSEED = 0

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score


# define path to save model
#model_path = 'multiclass_model_w1_30.h5'# This file then contains the already trained network, so that you don't have to retrain every time

## Data Ingestion


In [101]:
# read training data - It is the aircraft engine run-to-failure data.
from data_preprocessor import DataPreprocessor

# Initialize the preprocessor with the path to your training data file
preprocessor = DataPreprocessor('PM_train.txt')

In [102]:
train_df = pd.read_csv('PM_train.txt', sep=" ", header=None)
train_df.shape

(20631, 28)

`train_df.sort_values(['id','cycle'])`: This line sorts the DataFrame **train_df** first by the 'id' column and then by the 'cycle' column. It ensures that the data is ordered by engine ID and cycle number, which may be necessary for certain analyses or modeling tasks. The sorted DataFrame is then assigned back to the variable **train_df**.

## Data Preprocessing
data preprocessing step, particularly for labeling the data for training purposes. Let's break down what each part of the code does:


In [10]:
seq_array,dummy_label_array,seq_array_validation, dummy_label_array_validation,seq_array_test,dummy_label_array_test, sequence_cols = preprocessor.preprocess()
seq_array.shape, dummy_label_array.shape, seq_array_validation.shape, dummy_label_array_validation.shape,seq_array_test.shape, dummy_label_array_test.shape

((12138, 50, 25),
 (12138, 3),
 (1742, 50, 25),
 (1742, 3),
 (1751, 50, 25),
 (1751, 3))

In [11]:
print(sequence_cols)

['setting1', 'setting2', 'setting3', 'cycle_norm', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']


In [12]:
validation_df,test_df,train_df = preprocessor.test_data_pdmPolicy()
test_df.shape, train_df.shape, validation_df.shape

((2251, 29), (16138, 29), (2242, 29))

In [13]:
pdm_df = pd.concat([validation_df, test_df], ignore_index=True)
complete_df = pd.concat([train_df, pdm_df], ignore_index=True)
pdm_df.shape, pdm_df.columns

((4493, 29),
 Index(['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
        's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
        's15', 's16', 's17', 's18', 's19', 's20', 's21', 'RUL', 'label1',
        'label2'],
       dtype='object'))

In [14]:
pdm_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,RUL,label1,label2
0,81,1,-0.0050,0.0003,100.0,518.67,642.04,1589.91,1406.63,14.62,...,8.4455,0.03,391,2388,100.0,38.87,23.3365,239,0,0
1,81,2,0.0023,0.0002,100.0,518.67,642.65,1586.25,1407.88,14.62,...,8.4573,0.03,392,2388,100.0,38.91,23.3452,238,0,0
2,81,3,-0.0005,0.0005,100.0,518.67,642.55,1586.42,1396.40,14.62,...,8.4522,0.03,394,2388,100.0,39.04,23.3610,237,0,0
3,81,4,-0.0001,-0.0000,100.0,518.67,642.41,1594.89,1404.86,14.62,...,8.4403,0.03,392,2388,100.0,38.77,23.4206,236,0,0
4,81,5,0.0024,0.0002,100.0,518.67,643.41,1590.49,1409.58,14.62,...,8.3971,0.03,392,2388,100.0,39.04,23.3311,235,0,0


In [15]:
cols_normalize_train,cols_normalize_validation,cols_normalize_test = preprocessor.normalize_data_pdmPolicy()
print(cols_normalize_train)

Index(['cycle_norm', 's1', 's10', 's11', 's12', 's13', 's14', 's15', 's16',
       's17', 's18', 's19', 's2', 's20', 's21', 's3', 's4', 's5', 's6', 's7',
       's8', 's9', 'setting1', 'setting2', 'setting3'],
      dtype='object')


In [16]:
pdm_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,RUL,label1,label2
0,81,1,-0.0050,0.0003,100.0,518.67,642.04,1589.91,1406.63,14.62,...,8.4455,0.03,391,2388,100.0,38.87,23.3365,239,0,0
1,81,2,0.0023,0.0002,100.0,518.67,642.65,1586.25,1407.88,14.62,...,8.4573,0.03,392,2388,100.0,38.91,23.3452,238,0,0
2,81,3,-0.0005,0.0005,100.0,518.67,642.55,1586.42,1396.40,14.62,...,8.4522,0.03,394,2388,100.0,39.04,23.3610,237,0,0
3,81,4,-0.0001,-0.0000,100.0,518.67,642.41,1594.89,1404.86,14.62,...,8.4403,0.03,392,2388,100.0,38.77,23.4206,236,0,0
4,81,5,0.0024,0.0002,100.0,518.67,643.41,1590.49,1409.58,14.62,...,8.3971,0.03,392,2388,100.0,39.04,23.3311,235,0,0


>`Data Labeling`: This part calculates the Remaining Useful Life (RUL) or Time to Failure for each engine by finding the maximum cycle number (cycle) for each engine ID (id). The result is stored in a DataFrame rul with columns 'id' and 'max'.<br>

>`Merge RUL with Training Data`:the RUL information is merged back into the original training DataFrame **train_df** based on the engine ID. This allows each row in train_df to have the corresponding maximum cycle number as well.<br>

>`Calculate RUL`: This line calculates the RUL by subtracting the current cycle number ('cycle') from the maximum cycle number ('max') for each engine. This represents how many more cycles the engine is expected to operate before failure.<br>
>`Drop Unnecessary Columns`: After calculating RUL, the 'max' column, which was used temporarily to calculate RUL, is dropped from the DataFrame as it's no longer needed.<br>

> `Labeling for Classification`: This part assigns labels to each data point based on the calculated RUL. It defines thresholds `w1` and `w0`, and assigns:
>> * Label 1 ('label1') as 1 if RUL is less than or equal to 'w1', and 0 otherwise.
>> * Label2 ('label2') as 1 if RUL is less than or equal to 'w1', 2 if RUL is less than or equal to 'w0', and 0 otherwise.

 Now I want to separate the train_df set into a training/validation/test set. I will use 80% training sets for the training and 10% training sets as validation sets for hyperparameter tuning and the remaining 10% as test set for the PdM policy.

I separate into training and validation and test set before any data scaling is performed

Perform the min max scaling on the training data and validation dataset
use min_max_scaler.fit_transform()

>`Create a copy of the cycle column`: This line creates a new column named 'cycle_norm' in the train_df DataFrame and initializes it with the values from the original 'cycle' column. This column will be normalized later.<br>
> `Select columns for normalization`: This line selects all columns from **train_df** except 'id', 'cycle', 'RUL', 'label1', and 'label2'. These columns are the ones that will undergo normalization.


> `Initialize MinMaxScaler`: This line initializes a MinMaxScaler object from the scikit-learn preprocessing module. This scaler will be used to perform Min-Max normalization.<br>
> `Perform Min-Max normalization`: This line applies Min-Max normalization to the selected columns (`cols_normalize`) of the `train_df` DataFrame.<br>
> `min_max_scaler.fit_transform(train_df[cols_normalize])` computes the Min-Max normalization for the selected columns.<br>
> The resulting normalized values are stored in a new DataFrame called `norm_train_df`, with the same index as `train_df`.

> `Join normalized DataFrame with the original DataFrame`: This line joins the normalized DataFrame (`norm_train_df`) with the original DataFrame (`train_df`) excluding the columns that were normalized.<br>
> The resulting DataFrame `join_df` contains both the normalized columns and the original columns that were not normalized.


`Reorder columns`:
> * This line reorders the columns of `join_df` to match the original order of columns in `train_df`.
> * The reordered DataFrame is then assigned back to `train_df`, effectively replacing the original DataFrame with the normalized version.






## Vanilla Transformer

When you use `sequence_cols.extend(sensor_cols)`, it adds each element of `sensor_cols` to the end of `sequence_cols`.<br>
After this operation, `sequence_cols` will contain 25 elements: 4 operational settings followed by 21 sensor readings.



## generate sequences for each engine
> * This creates a generator expression that iterates over unique engine IDs in the training data.<br>
> * For each engine, it generates sequences using the `gen_sequence` function defined earlier.<br>
> * Each sequence is a list of sensor data, and multiple sequences are generated for each engine.<br>



> * This concatenates all the generated sequences into a single numpy array.
> * It converts the array to `float32` data type.
> * The resulting `seq_array` contains the sequences of sensor data, with shape `(num_sequences, sequence_length, num_features)`.


In [17]:
# we always take the measurements of the last 50 cycles as input!
# Every sequence is reduced by a length of 50 (=sequence_length). We have 80 training sets, 80*50 = 4000 "less" inputs
# train_df.shape = (16138, 30)
# seq_array.shape = (12138, 50, 25)

`Function Signature:` This function efficiently generates labels for each sequence of sensor data. It ensures that the labels are correctly aligned with the sequences and handles the special case where the first sequence uses the last label as its target.





> This function takes three arguments:
>> * `id_df:` DataFrame containing data for a specific engine (id).<br>
>> * `seq_length`: Length of the sequence window.<br>
>> * `label`: List of column names representing the labels.

`Data Preparation:`
> * `data_matrix = id_df[label].values:`
>> * This line extracts the columns specified by label from the DataFrame id_df and converts them to a numpy array.<br>
>> * It selects only the relevant label(s) needed for generating sequences.<br>

`Label Generation:`
> * `num_elements:`This line calculates the number of rows (elements) in the data matrix, which corresponds to the number of labels.<br>
> * `return data_matrix[seq_length:num_elements, :]:`
>> * This line returns the labels associated with each sequence.<br>
>> * It removes the first `seq_length` labels because, for each engine (`id`), the first sequence of size `seq_length` uses the last label as its target. The previous labels are discarded.<br>
>> * All subsequent sequences for the same engine (`id`) will have one label associated with them step by step.<br>








When modeling multi-class classification problems using neural networks,<br>
it is good practice to reshape the output attribute from a vector that contains values for each class value to be<br>
a matrix with a boolean for each class value and whether or not a given instance has that class value or not.<br>
This is called one hot encoding or creating dummy variables from a categorical variable.<br>

from tensorflow.keras.utils import to_categorical<br>

`to_categorical` is a utility function in Keras that converts class vectors (integers) to binary class matrices.<br>
`dummy_label_array = to_categorical(label_array):`This line applies one-hot encoding to the `label_array`.<br>
`label_array` contains the labels associated with each sequence, where each label represents a class or category.<br>
> * One-hot encoding converts these integer labels into binary vectors, where each vector has a length equal to the number of classes and contains a 1 in the position corresponding to the class and 0s elsewhere.





In [18]:
dummy_label_array

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [19]:
dummy_label_array_validation

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [20]:
dummy_label_array_validation.shape

(1742, 3)

In [21]:
dummy_label_array.shape

(12138, 3)

In [22]:
nb_features = seq_array.shape[2]
nb_out      = dummy_label_array.shape[1]
nb_features, nb_out

(25, 3)

`Extracting Feature and Output Dimensions:`
> `nb_features:`Determines the number of features in the input sequence data.<br>
> `nb_out:`Determines the number of output classes. It's extracted from the shape of the label array.<br>

`Defining the Model Architecture:` describe in the code below.
`Compiling the Model:` `model.compile(...)` Here, `categorical_crossentropy` is used as the loss function for multi-class classification.

`Model Summary:`Prints a summary of the model architecture, including the layers and their parameters.

`Training the Model:` `model.fit(...):` Trains the model on the training data. It specifies the input data (`seq_array`) and the corresponding labels (`dummy_label_array`). Other parameters include the number of epochs, batch size, validation split, verbosity, and callbacks.<br>


`history.history.keys():` After training, this prints the keys of the history object, which contains information about training and validation metrics over each epoch.



### Define the Dataset:
Create a custom dataset class to handle your multivariate time series data with labels.




In [23]:
#from google.colab import drive
#drive.mount('/content/drive')

#import os
#os.chdir('/content/drive/MyDrive/Colab Notebooks/LSTM_Antonis')

`Shuffling Batches:` By setting `shuffle=True` in the `DataLoader` for the training set, you ensure that the order of batches is shuffled each epoch. This maintains the temporal structure within each batch while still introducing variability in the order in which batches are processed.<br>

`DataLoader for Validation:` Ensure `shuffle=False` for the validation set to maintain the sequence order during validation.

In [24]:
# Import custom classes
import torch
from torch.utils.data import TensorDataset, DataLoader

from dataset import MultivariateTimeSeriesDataset
from model import TransformerTimeSeriesModel

# Load initial parameters from params.json
with open('params.json', 'r') as f:
    params = json.load(f)

# Load hyperparameter search space from hyperparameters.json
with open('hyperparameters.json', 'r') as f:
    hp_config = json.load(f)
    


# Example usage with seq_array and dummy_var
#seq_array --> (12138, 50, 25)  # use this as your actual data
#dummy_var --->  (12138, 3) # use this as your actual dummy variable (3 classes)

#train_dataset = MultivariateTimeSeriesDataset(seq_array, dummy_label_array, params['seq_length'])
#val_dataset = MultivariateTimeSeriesDataset(seq_array_validation, dummy_label_array_validation, params['seq_length'])

# Shuffle batches by setting shuffle=True in DataLoader
#train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)


In [25]:
"""
os.environ['WANDB_API_KEY'] = 'f05e720396a77affdb7e82b525ef1a912da7aaf4'

try:
    wandb.login()
    print(f"Successfully logged in as: {wandb.api.viewer()['entity']}")
except wandb.errors.AuthenticationError:
    print("Failed to authenticate. Please check your API key.")
"""


'\nos.environ[\'WANDB_API_KEY\'] = \'f05e720396a77affdb7e82b525ef1a912da7aaf4\'\n\ntry:\n    wandb.login()\n    print(f"Successfully logged in as: {wandb.api.viewer()[\'entity\']}")\nexcept wandb.errors.AuthenticationError:\n    print("Failed to authenticate. Please check your API key.")\n'

In [26]:
"""
current_entity = wandb.api.viewer()['entity']
print(f"Currently logged in as: {current_entity}")
if current_entity != "jitendratiwari11":
    raise ValueError(f"Logged in as {current_entity}, but expected to be logged in as jitendra")

print(wandb.api.viewer()['entity'])
"""

'\ncurrent_entity = wandb.api.viewer()[\'entity\']\nprint(f"Currently logged in as: {current_entity}")\nif current_entity != "jitendratiwari11":\n    raise ValueError(f"Logged in as {current_entity}, but expected to be logged in as jitendra")\n\nprint(wandb.api.viewer()[\'entity\'])\n'

In [27]:
#wandb.finish()

In [28]:
#print(wandb.config)

### Define the Transformer Model:
Implement the vanilla Transformer architecture, ensuring it takes the dummy variable as an input.



### Training the model

Define the training loop with the loss function and optimizer, and include the dummy variable in the forward pass.



In [29]:
"""
def train_and_evaluate(config, num_epochs,use_early_stopping=True):
    model = TransformerTimeSeriesModel(
        input_dim=params['input_dim'],
        seq_length=params['seq_length'],
        num_classes=params['num_classes'],
        model_dim=config['model_dim'],
        num_heads=config['num_heads'],
        num_layers=config['num_layers'],
        dropout_rate=config['dropout_rate']
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    scaler = GradScaler()

    # Set up datasets
    train_dataset = MultivariateTimeSeriesDataset(seq_array, dummy_label_array, params['seq_length'])
    val_dataset = MultivariateTimeSeriesDataset(seq_array_validation, dummy_label_array_validation, params['seq_length'])

    # Set up data loaders
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)
    
    train_losses = []
    val_losses = []
    val_accuracies = []

    best_val_loss = float('inf')
    best_val_accuracy = 0
    best_accuracy = 0
    best_model = None
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        for i, batch in enumerate(train_pbar):
            inputs, labels = batch
            inputs, labels = inputs.float().to(device), labels.float().to(device)
            
            with autocast():
                outputs_class = model(inputs)
                outputs_reshaped = outputs_class.view(-1, model.num_classes)
                labels_reshaped = labels.view(-1, model.num_classes).argmax(dim=1)
                loss = criterion(outputs_reshaped, labels_reshaped) / config['gradient_accumulation_steps']

            scaler.scale(loss).backward()
            
            if (i + 1) % config['gradient_accumulation_steps'] == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            total_train_loss += loss.item() * config['gradient_accumulation_steps']
            
            
            current_train_loss = total_train_loss / (i + 1)
            train_pbar.set_postfix({'train loss': current_train_loss})
            
            if i % 100 == 0:
                torch.cuda.empty_cache()
                gc.collect()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        
        #Validation loop
        model.eval()
        total_val_loss = 0.0
        all_preds = []
        all_labels = []

        
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
        with torch.no_grad():
            for batch in val_pbar:
                inputs, labels = batch
                inputs, labels = inputs.float().to(device), labels.float().to(device)
                
                with autocast():
                    outputs_class = model(inputs)
                    outputs_reshaped = outputs_class.view(-1, model.num_classes)
                    labels_reshaped = labels.view(-1, model.num_classes).argmax(dim=1)
                    val_loss = criterion(outputs_reshaped, labels_reshaped)
                
                total_val_loss += val_loss.item()

                _, preds = torch.max(outputs_class, dim=2)
                all_preds.extend(preds.cpu().numpy().reshape(-1))
                all_labels.extend(labels.argmax(dim=2).cpu().numpy().reshape(-1))

                current_val_loss = total_val_loss / (val_pbar.n + 1)
                val_pbar.set_postfix({'val loss': current_val_loss})

        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        accuracy = accuracy_score(all_labels, all_preds)
        val_accuracies.append(accuracy)
        
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Val Loss: {avg_val_loss:.4f}, "
              f"Val Accuracy: {accuracy:.2f}%")
        
        wandb.log({
            "epoch": epoch,
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "val_accuracy": accuracy
        })
        if use_early_stopping:
            # Early stopping check
            if accuracy > best_val_accuracy:
                best_val_accuracy = accuracy
                best_val_loss = avg_val_loss
                best_model = model.state_dict()
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1

            if epochs_without_improvement >= params['patience']:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
        else:
            # If not using early stopping, always update best model
            best_val_accuracy = accuracy
            best_val_loss = avg_val_loss
            best_model = model.state_dict()
            

        
        torch.cuda.empty_cache()
        gc.collect()

    #Load the best model before returning
    model.load_state_dict(best_model)

    return model, train_losses, val_losses, val_accuracies
"""

'\ndef train_and_evaluate(config, num_epochs,use_early_stopping=True):\n    model = TransformerTimeSeriesModel(\n        input_dim=params[\'input_dim\'],\n        seq_length=params[\'seq_length\'],\n        num_classes=params[\'num_classes\'],\n        model_dim=config[\'model_dim\'],\n        num_heads=config[\'num_heads\'],\n        num_layers=config[\'num_layers\'],\n        dropout_rate=config[\'dropout_rate\']\n    )\n\n    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n    model.to(device)\n    \n    criterion = nn.CrossEntropyLoss()\n    optimizer = optim.AdamW(model.parameters(), lr=config[\'learning_rate\'], weight_decay=config[\'weight_decay\'])\n    scaler = GradScaler()\n\n    # Set up datasets\n    train_dataset = MultivariateTimeSeriesDataset(seq_array, dummy_label_array, params[\'seq_length\'])\n    val_dataset = MultivariateTimeSeriesDataset(seq_array_validation, dummy_label_array_validation, params[\'seq_length\'])\n\n    # Set up data loaders\n

In [30]:
"""

def objective():
    global trial_counter
    with wandb.init() as run:
        config = wandb.config
        best_model, train_losses, val_losses, val_accuracies = train_and_evaluate(config, num_epochs=config.num_epochs)
        
        best_val_accuracy = max(val_accuracies)
        wandb.log({"best_val_accuracy": best_val_accuracy})
        
        trial_counter = run.step + 1
        print(f"Trial {trial_counter}/{hp_config['num_trials']}, "
              f"Best Val Accuracy: {best_val_accuracy:.2f}")

        
        # Save the best model
        torch.save(best_model.state_dict(), 'best_model_3_class.pth')
        wandb.save('best_model_3_class.pth')
        
        return best_val_accuracy

"""


'\n\ndef objective():\n    global trial_counter\n    with wandb.init() as run:\n        config = wandb.config\n        best_model, train_losses, val_losses, val_accuracies = train_and_evaluate(config, num_epochs=config.num_epochs)\n        \n        best_val_accuracy = max(val_accuracies)\n        wandb.log({"best_val_accuracy": best_val_accuracy})\n        \n        trial_counter = run.step + 1\n        print(f"Trial {trial_counter}/{hp_config[\'num_trials\']}, "\n              f"Best Val Accuracy: {best_val_accuracy:.2f}")\n\n        \n        # Save the best model\n        torch.save(best_model.state_dict(), \'best_model_3_class.pth\')\n        wandb.save(\'best_model_3_class.pth\')\n        \n        return best_val_accuracy\n\n'

In [31]:
"""
# Define the hyperparameter search space
sweep_config = {
    'method': hp_config['method'],
    'metric': {'name': hp_config['metric_name'], 'goal': hp_config['metric_goal']},
    'parameters': {
        'model_dim': {'values': hp_config['model_dim']},
        'num_heads': {'values': hp_config['num_heads']},
        'num_layers': {'values': hp_config['num_layers']},
        'dropout_rate': {'distribution': 'uniform', 'min': hp_config['dropout_rate']['min'], 'max': hp_config['dropout_rate']['max']},
        'learning_rate': {'distribution': 'log_uniform_values', 'min': hp_config['learning_rate']['min'], 'max': hp_config['learning_rate']['max']},
        'batch_size': {'values': hp_config['batch_size']},
        'num_epochs': {'values': hp_config['num_epochs']},
        'weight_decay': {'distribution': 'log_uniform_values', 'min': hp_config['weight_decay']['min'], 'max': hp_config['weight_decay']['max']},
        'gradient_accumulation_steps': {'values': hp_config['gradient_accumulation_steps']}
    }
}


#sweep_id = wandb.sweep(sweep_config, project="transformer_time_series", entity="jitendratiwari11")
sweep_id= 'c3hgzsib'
# Start the sweep
#wandb.agent(sweep_id, function=objective, count=hp_config['num_trials'])
wandb.agent(sweep_id, function=objective, count=hp_config['num_trials'], project="transformer_time_series", entity="jitendratiwari11")

# After all trials, find the best run
api = wandb.Api()
runs = api.runs("jitendratiwari11/transformer_time_series")
best_run = max(runs, key=lambda run: run.summary.get('best_val_accuracy', 0))

print(f"Best run: {best_run.name}")
print(f"Best validation accuracy: {best_run.summary['best_val_accuracy']}")
print("Best parameters:")
print(json.dumps(best_run.config, indent=4))

# Update params.json with best parameters
params.update(best_run.config)
with open('params.json', 'w') as f:
    json.dump(params, f, indent=4)

print("Best parameters saved to params.json")

# Run the best model for longer epochs
print("Running best model for longer epochs...")
wandb.init(project="transformer_time_series", name="best_model_long_run", config=params, reinit=True)

best_model, train_losses, val_losses, val_accuracies = train_and_evaluate(params, num_epochs=100)



"""

'\n# Define the hyperparameter search space\nsweep_config = {\n    \'method\': hp_config[\'method\'],\n    \'metric\': {\'name\': hp_config[\'metric_name\'], \'goal\': hp_config[\'metric_goal\']},\n    \'parameters\': {\n        \'model_dim\': {\'values\': hp_config[\'model_dim\']},\n        \'num_heads\': {\'values\': hp_config[\'num_heads\']},\n        \'num_layers\': {\'values\': hp_config[\'num_layers\']},\n        \'dropout_rate\': {\'distribution\': \'uniform\', \'min\': hp_config[\'dropout_rate\'][\'min\'], \'max\': hp_config[\'dropout_rate\'][\'max\']},\n        \'learning_rate\': {\'distribution\': \'log_uniform_values\', \'min\': hp_config[\'learning_rate\'][\'min\'], \'max\': hp_config[\'learning_rate\'][\'max\']},\n        \'batch_size\': {\'values\': hp_config[\'batch_size\']},\n        \'num_epochs\': {\'values\': hp_config[\'num_epochs\']},\n        \'weight_decay\': {\'distribution\': \'log_uniform_values\', \'min\': hp_config[\'weight_decay\'][\'min\'], \'max\': hp_con

In [32]:
"""
# Run the best model for longer epochs
print("Running best model for longer epochs...")
wandb.init(project="transformer_time_series", name="experiment2", config=params, reinit=True)

best_model, train_losses, val_losses, val_accuracies = train_and_evaluate(params, num_epochs=100, use_early_stopping=False)

# Save the model after 100 epochs
torch.save(best_model.state_dict(), 'best_model_after_100_epochs.pth')
print("Model after 100 epochs saved to best_model_after_100_epochs.pth")
"""


'\n# Run the best model for longer epochs\nprint("Running best model for longer epochs...")\nwandb.init(project="transformer_time_series", name="experiment2", config=params, reinit=True)\n\nbest_model, train_losses, val_losses, val_accuracies = train_and_evaluate(params, num_epochs=100, use_early_stopping=False)\n\n# Save the model after 100 epochs\ntorch.save(best_model.state_dict(), \'best_model_after_100_epochs.pth\')\nprint("Model after 100 epochs saved to best_model_after_100_epochs.pth")\n'

## Model Evaluation on Validation set created during preprocessing of data

In [33]:
"""
# Plot the losses and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Validation Accuracy')

plt.tight_layout()
plt.savefig('learning_curves_100_epochs.png')
plt.close()

wandb.log({
    "final_train_loss": train_losses[-1],
    "final_val_loss": val_losses[-1],
    "final_val_accuracy": val_accuracies[-1],
    "training_curves": wandb.Image('training_curves.png')
})

print(f"Final train loss: {train_losses[-1]:.4f}")
print(f"Final validation loss: {val_losses[-1]:.4f}")
print(f"Final validation accuracy: {val_accuracies[-1]:.2f}%")

wandb.finish()
"""


'\n# Plot the losses and accuracy\nplt.figure(figsize=(12, 4))\nplt.subplot(1, 2, 1)\nplt.plot(train_losses, label=\'Train Loss\')\nplt.plot(val_losses, label=\'Validation Loss\')\nplt.xlabel(\'Epoch\')\nplt.ylabel(\'Loss\')\nplt.legend()\nplt.title(\'Training and Validation Loss\')\n\nplt.subplot(1, 2, 2)\nplt.plot(val_accuracies, label=\'Validation Accuracy\')\nplt.xlabel(\'Epoch\')\nplt.ylabel(\'Accuracy (%)\')\nplt.legend()\nplt.title(\'Validation Accuracy\')\n\nplt.tight_layout()\nplt.savefig(\'learning_curves_100_epochs.png\')\nplt.close()\n\nwandb.log({\n    "final_train_loss": train_losses[-1],\n    "final_val_loss": val_losses[-1],\n    "final_val_accuracy": val_accuracies[-1],\n    "training_curves": wandb.Image(\'training_curves.png\')\n})\n\nprint(f"Final train loss: {train_losses[-1]:.4f}")\nprint(f"Final validation loss: {val_losses[-1]:.4f}")\nprint(f"Final validation accuracy: {val_accuracies[-1]:.2f}%")\n\nwandb.finish()\n'

In [34]:
#print(f" total nr of parameters as per best model: {sum(p.numel() for p in best_model.parameters() if p.requires_grad)}")

## Confusion matricx and Classification report

For each test set, I need to give the on-line sensor data as input to the trained Transformer.


In [35]:
import torch
from model import TransformerTimeSeriesModel
from sklearn.metrics import confusion_matrix, classification_report


# define path to save model
#model_path = 'best_model_3_class.pth'# This file then contains the already trained network, so that you don't have to retrain every time
model_path = 'best_model_after_100_epochs.pth'# This file then contains the already trained network, so that you don't have to retrain every time


# Check if the model file exists
if os.path.isfile(model_path):
    estimator = TransformerTimeSeriesModel(
    input_dim=params['input_dim'],
        seq_length=params['seq_length'],
        num_classes=params['num_classes'],
        model_dim=params['model_dim'],
        num_heads=params['num_heads'],
        num_layers=params['num_layers'],
        dropout_rate=params['dropout_rate']
    )

    # Load the state dict
    estimator.load_state_dict(torch.load(model_path))
    
    print(f"Model loaded from {model_path}")
    print(f" total nr of parameters as per best model: {sum(p.numel() for p in estimator.parameters() if p.requires_grad)}")
    
    
    # Move the model to the appropriate device (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    estimator = estimator.to(device)

    # Set the model to evaluation mode
    estimator.eval()

    test_dataset = MultivariateTimeSeriesDataset(seq_array_test, dummy_label_array_test, params['seq_length'])
    
    test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)
    
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = estimator(inputs)
            
            _, preds = torch.max(outputs, dim=2)
            all_preds.extend(preds.cpu().numpy().reshape(-1))
            all_labels.extend(labels.argmax(dim=2).cpu().numpy().reshape(-1))

            
           # _, preds = torch.max(outputs, 1)
            #all_preds.extend(preds.cpu().numpy())
            #all_labels.extend(torch.argmax(labels, dim=1).cpu().numpy())
            
    # Convert lists to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Optionally, print a classification report
    report = classification_report(all_labels, all_preds)
    print("Classification Report:")
    print(report)

else:
    print(f"No saved model found at {model_path}")


Model loaded from best_model_after_100_epochs.pth
 total nr of parameters as per best model: 32051
Confusion Matrix:
[[67150  4900     0]
 [    0  6849  3151]
 [    0     0  5500]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.96     72050
           1       0.58      0.68      0.63     10000
           2       0.64      1.00      0.78      5500

    accuracy                           0.91     87550
   macro avg       0.74      0.87      0.79     87550
weighted avg       0.93      0.91      0.91     87550



In [36]:
# Assumptions for the costs, taken by the 2019 RESS paper
C_p    = 100
C_c    = 1000
C_unav = 10
C_inv  = 1
DT     = 10  # Decisions can be taken every DT=10
L      = 20  # lead time

In [37]:
array_decisions = np.arange(0,400,DT) # decisions can only be made every DT = 10 cycles
print(array_decisions)

[  0  10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170
 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350
 360 370 380 390]


In [38]:
# estimator.predict(seq_array_validation_k).reshape(3) returns a vector with 3 elements
# [Pr(RUL>w1), Pr(w0<RUL<=w1), Pr(RUL<=w0)]

In [39]:
pdm_df['cycle_norm'] = pdm_df['cycle']
pdm_df.shape

(4493, 30)

In [40]:
train_df['cycle_norm'] = train_df['cycle']
train_df.shape

(16138, 30)

In [41]:
#result_df['cycle_norm'] = result_df['cycle']
#result_df.head(193)

## PdM policy evaluation on a the whole (test data+ validation) set (ids 81 to 101)


In [42]:
#costs_rep_array   = np.zeros(10) # An array to store costs related to replacements.

#costs_delay_array = np.zeros(10) # An array to store costs related to delays.
#costs_stock_array = np.zeros(10) # An array to store costs related to stock.

#t_LC_array        = np.zeros(10) # An array to store lead time.
#t_order_array     = np.zeros(10) # An array to store order time.

> 1. Initializes a counter variable to 0.
> 2. Iterates over unique IDs in the `test_df` DataFrame.
> 3. For each ID:
>> * Sets flags for preventive replacement and ordering to False.<br>
>> * Iterates over cycles within the range of the DataFrame.<br>
>> * Checks if the current cycle is in the `array_decisions`.<br>
>> * If it is, preprocesses the validation data for the Transformer model.<br>
>> * Predicts the probability of RUL being smaller than w1 and DT (decision time) using the trained model.<br>
>> * Evaluates decision heuristics:
>>> * If no order has been placed yet and the cost of preventive replacement is less than or equal to the cost of waiting until `w1`, orders the component and sets the order time.<br>
>>> * If the cost of preventive replacement is less than or equal to the cost of waiting until `DT`, performs preventive replacement, calculates related costs, and breaks the loop.<br>
>> If preventive replacement is not performed:
>>> * Sets the component failure time to the last cycle in the ID's data.<br>
>>> * Sets replacement costs to `C_c`.<br>
>>> * Calculates delay costs based on whether an order has been placed.<br>
>> * Prints diagnostic information for each iteration.
>> * Increments the counter.


This code essentially simulates a decision-making process for component maintenance based on predictive models and cost considerations.








In [43]:
"""
from sklearn import preprocessing

counter = 0
for id in test_df['id'].unique():
    print('ID:', id)
    preventive_replacement = False
    order                  = False

    for cycle in range(test_df[test_df['id']==id].shape[0]-params['seq_length']+1):

        if cycle in array_decisions:
            min_max_scaler = preprocessing.MinMaxScaler()
            norm_test_df = pd.DataFrame(min_max_scaler.fit_transform(test_df[test_df['id']==id][cols_normalize_train][:params['seq_length']+cycle]),
                 columns=cols_normalize_train,
                 index=test_df[test_df['id']==id][:params['seq_length']+cycle].index)

            join_df = test_df[test_df['id']==id][:params['seq_length']+cycle][test_df[test_df['id']==id][:params['seq_length']+cycle].columns.difference(cols_normalize_train)].join(norm_test_df)
            test_df_eval_online = join_df.reindex(columns = test_df[test_df['id']==id][cycle:params['seq_length']+cycle].columns)

            seq_array_test_k = test_df_eval_online[sequence_cols].values[cycle:params['seq_length']+cycle]
            seq_array_test_k = np.asarray(seq_array_test_k).astype(np.float32).reshape(1,params['seq_length'], nb_features)
            #prob_RUL_smaller_DT    = estimator.predict(seq_array_test_k).reshape(3)[2]
            #prob_RUL_smaller_w1    = estimator.predict(seq_array_test_k).reshape(3)[1]
            
            prob_RUL_smaller_DT    = estimator(seq_array_test_k).reshape(3)[2]
            prob_RUL_smaller_w1    = estimator(seq_array_test_k).reshape(3)[1]

            print('prob_RUL_smaller_w1:', prob_RUL_smaller_w1)
            print('prob_RUL_smaller_DT:', prob_RUL_smaller_DT)


            # evaluate decision heuristics
            if order == False:
                if C_p <= prob_RUL_smaller_w1*C_c:
                    print('prob_RUL_smaller_w1:', prob_RUL_smaller_w1)
                    print('prob_RUL_smaller_DT:', prob_RUL_smaller_DT)
                    t_order_array[counter] = params['seq_length']+cycle
                    order = True
                    print('component ordering at cycle:', t_order_array[counter])

            if C_p <= prob_RUL_smaller_DT*C_c:
                print('prob_RUL_smaller_DT:', prob_RUL_smaller_DT)

                t_LC_array[counter] = params['seq_length']+cycle
                costs_rep_array[counter] = C_p
                print('preventive replacement informed at cycle:', t_LC_array[counter])
                # print('component lifecycle:', t_LC)
                preventive_replacement = True
                costs_delay_array[counter] = max(t_order_array[counter]+L-t_LC_array[counter], 0) * C_unav

                costs_stock_array[counter]  = max(t_LC_array[counter] -(t_order_array[counter]+L), 0)*C_inv
                # print('delay time', max(t_order+L-t_LC, 0))
                # print('cost_delay_id:',cost_delay_id)
                # print('cost of stock:', cost_stock_id)
                break

    if preventive_replacement == False:
        t_LC_array[counter] = test_df[test_df['id']==id]['cycle'].iloc[-1]
        print('Component failure at t:', t_LC_array[counter])
        costs_rep_array[counter] = C_c

        if order == False:
            costs_delay_array[counter] = L * C_unav
        else:
            costs_delay_array[counter] = max(t_order_array[counter]+L-t_LC_array[counter], 0) * C_unav
            costs_stock_array[counter] = max(t_LC_array[counter] -(t_order_array[counter]+L), 0)*C_inv

    print('True failure:', test_df[test_df['id']==id]['cycle'].iloc[-1])
    print('-----------------------------------------')
    counter+=1

"""

"\nfrom sklearn import preprocessing\n\ncounter = 0\nfor id in test_df['id'].unique():\n    print('ID:', id)\n    preventive_replacement = False\n    order                  = False\n\n    for cycle in range(test_df[test_df['id']==id].shape[0]-params['seq_length']+1):\n\n        if cycle in array_decisions:\n            min_max_scaler = preprocessing.MinMaxScaler()\n            norm_test_df = pd.DataFrame(min_max_scaler.fit_transform(test_df[test_df['id']==id][cols_normalize_train][:params['seq_length']+cycle]),\n                 columns=cols_normalize_train,\n                 index=test_df[test_df['id']==id][:params['seq_length']+cycle].index)\n\n            join_df = test_df[test_df['id']==id][:params['seq_length']+cycle][test_df[test_df['id']==id][:params['seq_length']+cycle].columns.difference(cols_normalize_train)].join(norm_test_df)\n            test_df_eval_online = join_df.reindex(columns = test_df[test_df['id']==id][cycle:params['seq_length']+cycle].columns)\n\n            seq_

# PDM Policy1

1. `Policy Overview:` We first consider the simple dynamic PdM decision setting, in which one determines at each time step
    tk whether a component should be preventively replaced or not. The assumption here is that the new
    component is readily available when a preventive replacement is decided or a corrective replacement is
    imposed. <br>
2. `Decision Making Process:`
> At each time step tk = k * ΔT, the policy decides whether to take action or not.<br>
> The action arep,k can be either:
>> a) DN (Do Nothing) <br>
>> b) PR (Preventive Replacement)<br>

3. `Decision Rule:`
> If Pr(RULpred,k ≤ ΔT) < pthres, then Do Nothing (DN) <br>
> Otherwise, perform Preventive Replacement (PR) <br>
where:
> RULpred,k is the predicted Remaining Useful Life at time tk <br>
> ΔT is the time step <br>
> pthres is a variable heuristic threshold <br>
4. `Threshold Determination:`
> Initially, pthres is set to cp/cc <br>
> cp is the cost of preventive replacement <br>
> cc is the cost of component failure <br>
5. `Cost Considerations:`
> PR action costs cp <br>
> DN action risks a potential cost of Pr(RULpred,k ≤ ΔT) * cc <br>
6. `Rationale:`
> PR is performed only when its cost is less than the predicted risk of failure in the next time step. <br>
7. `Input Requirements:`
>  `current_cycle:` variable represents the current time step or cycle within the sequence of data for a specific engine or component. It is used to iterate through the sequence of cycles for each unique engine ID in the dataset. <br>
>  The policy needs Pr(RULpred,k ≤ ΔT) from the prognostic algorithm. <br>
8. `Outcome:`
> The policy informs replacement decisions for each component. <br>
> It determines C_rep(i) (replacement cost) and Tlc(i) (lifecycle time) for each component i. <br>

>> 1. `t_LC_array(Lifecycle Time) :`
>>> * This array represents `Tlc(i) = min[T_R(i), T_F(i)]` for each component. <br>
>>> * In the code, it's set in two cases:
>>> a) When a preventive replacement is decided: `t_LC_array[counter] = params['seq_length'] + current_cycle`
>>> b) When no preventive replacement occurred (implying failure): `t_LC_array[counter] = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]`
>>> * This aligns with the definition of `Tlc(i)` being the minimum of preventive replacement time or failure time.
>> 2. `costs_rep_array (Replacement Cost):`
>>> * This array represents C_rep(i) for each component.
>>> * In the code, it's set as follows:
>>> a) For preventive replacement: `costs_rep_array[counter] = C_p` <br>
>>> b) For corrective replacement (when no preventive replacement occurred): `costs_rep_array[counter] = C_c` <br>
>>> This directly implements the condition as mentioned in equation 8: `C_rep(i) = (cp, if T_R(i) < T_F(i), cc, else.`


9. `t_order_array:`
> * Meaning: This array stores the cycle times at which components are ordered. <br>
> * Significance: It helps track when preventive maintenance actions are initiated, allowing for analysis of the timing of maintenance decisions. <br>

10. `t_LC_array:`
> Meaning: This array likely stores the lifecycle times for each component. <br>
> Significance: It represents either the time of preventive replacement or the time of failure for each component, which is crucial for evaluating the effectiveness of the maintenance policy. <br>

11. `costs_rep_array:`
> * Meaning: This array stores the replacement costs for each component. <br>
> * Significance: It captures the financial impact of replacements, whether they are preventive (C_p) or corrective (C_c). This is essential for assessing the cost-effectiveness of the maintenance strategy. <br>

12. `costs_delay_array:`
> * Meaning: This array stores the costs associated with delays in component replacement. <br>
> * Significance: It represents the financial penalties incurred when a component fails before a replacement arrives, helping to quantify the impact of maintenance timing on overall costs. <br>

13. `costs_stock_array:`
> Meaning: This array stores the costs related to holding replacement components in stock. <br>
> Significance: It captures the inventory holding costs when components are ordered too early, balancing the trade-off between early ordering to prevent failures and the costs of storing components. <bR>



In [44]:
pdm_df.head(136)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,81,1,-0.0050,0.0003,100.0,518.67,642.04,1589.91,1406.63,14.62,...,0.03,391,2388,100.0,38.87,23.3365,239,0,0,1
1,81,2,0.0023,0.0002,100.0,518.67,642.65,1586.25,1407.88,14.62,...,0.03,392,2388,100.0,38.91,23.3452,238,0,0,2
2,81,3,-0.0005,0.0005,100.0,518.67,642.55,1586.42,1396.40,14.62,...,0.03,394,2388,100.0,39.04,23.3610,237,0,0,3
3,81,4,-0.0001,-0.0000,100.0,518.67,642.41,1594.89,1404.86,14.62,...,0.03,392,2388,100.0,38.77,23.4206,236,0,0,4
4,81,5,0.0024,0.0002,100.0,518.67,643.41,1590.49,1409.58,14.62,...,0.03,392,2388,100.0,39.04,23.3311,235,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,81,132,0.0018,0.0002,100.0,518.67,642.54,1595.33,1413.15,14.62,...,0.03,393,2388,100.0,38.82,23.2471,108,0,0,132
132,81,133,0.0024,0.0005,100.0,518.67,642.55,1592.20,1406.65,14.62,...,0.03,393,2388,100.0,38.89,23.3028,107,0,0,133
133,81,134,0.0008,-0.0004,100.0,518.67,642.62,1589.66,1405.98,14.62,...,0.03,395,2388,100.0,39.03,23.2205,106,0,0,134
134,81,135,0.0009,0.0002,100.0,518.67,642.60,1590.30,1407.55,14.62,...,0.03,394,2388,100.0,38.92,23.3995,105,0,0,135


In [45]:
pdm_df.shape

(4493, 30)

1. `Sequence Data Preparation: `<br>
 * When you prepare a sequence for prediction, you're using data from `current_cycle` to `current_cycle + params['seq_length']`. <br>
 * This means that the sequence includes data points from the `current_cycle` up to `params['seq_length']` cycles into the future. <br>
 

2. `Model Prediction: ` <br>
 * When your model makes a prediction based on this sequence, it's essentially predicting the state at the end of the sequence.<br>
 * The prediction is about the condition at cycle `current_cycle + params['seq_length']`, not at `current_cycle`. <br>
 
3. `Decision Point: `<br>
 * The decision to perform a preventive replacement (PR) is made based on the prediction for the end of the sequence.<br>
 * This decision point is effectively at `current_cycle + params['seq_length']`.<br>
4. `Realistic Timing: `<br>
 * In a real-world scenario, you would make the decision to replace a component based on its predicted future state, not its current state.<br>
 * Setting `T_R_i = current_cycle + params['seq_length']` reflects this reality - you're deciding to replace at the future point for which you've made the prediction.
 
5. `Consistency with Model Output: `<br>
 * Your model is trained to predict the state at the end of each input sequence.<br>
 * By setting `T_R_i` to the end of the sequence, you're aligning the replacement decision with what the model is actually predicting.<br>
 
6. `Avoiding Premature Decisions: `<br>
 * If you set `T_R_i = current_cycle`, you might be making decisions based on incomplete information.<br>
 * The full sequence might show trends or patterns that only become clear by the end of the `params['seq_length']` cycles.<br>

Example: <br>
Let's say `current_cycle = 100` and `params['seq_length'] = 50`:
 * Your sequence data covers cycles 100 to 150.<br>
 * The model predicts the state at cycle 150.<br>
 * If the prediction indicates a need for replacement, it's logical to set `T_R_i = 150` (which is `current_cycle + params['seq_length']`), not 100.


## Implementing PDM policy 1 without ordering

In [46]:
"""
def prepare_sequence_data(pdm_df, id, current_cycle, params, cols_normalize_train, sequence_cols):
    min_max_scaler = preprocessing.MinMaxScaler()
    norm_pdm_df = pd.DataFrame(min_max_scaler.fit_transform(pdm_df[pdm_df['id'] == id][cols_normalize_train][:params['seq_length'] + current_cycle]),
                                columns=cols_normalize_train,
                                index=pdm_df[pdm_df['id'] == id][:params['seq_length'] + current_cycle].index)
    #print("shape of norm_pdm_df: ", norm_pdm_df.shape)

    join_df = pdm_df[pdm_df['id'] == id][:params['seq_length'] + current_cycle][pdm_df[pdm_df['id'] == id][:params['seq_length'] + current_cycle].columns.difference(cols_normalize_train)].join(norm_pdm_df)
    #print("shape of joint_df: ",join_df.shape)
    pdm_df_eval_online = join_df.reindex(columns=pdm_df[pdm_df['id'] == id][current_cycle:params['seq_length'] + current_cycle].columns)
    
    #print("shape of pdm_df_eval_online: ",pdm_df_eval_online.shape)
    seq_array_test_k = pdm_df_eval_online[sequence_cols].values[current_cycle:params['seq_length'] + current_cycle]
    #print("shape of seq_array_test_k before reshaping: ",seq_array_test_k.shape)
    seq_array_test_k = np.asarray(seq_array_test_k).astype(np.float32).reshape(1, params['seq_length'], len(sequence_cols))
    #print("shape of seq_array_test_k after reshaping: ",seq_array_test_k.shape)
    
    return seq_array_test_k

"""

'\ndef prepare_sequence_data(pdm_df, id, current_cycle, params, cols_normalize_train, sequence_cols):\n    min_max_scaler = preprocessing.MinMaxScaler()\n    norm_pdm_df = pd.DataFrame(min_max_scaler.fit_transform(pdm_df[pdm_df[\'id\'] == id][cols_normalize_train][:params[\'seq_length\'] + current_cycle]),\n                                columns=cols_normalize_train,\n                                index=pdm_df[pdm_df[\'id\'] == id][:params[\'seq_length\'] + current_cycle].index)\n    #print("shape of norm_pdm_df: ", norm_pdm_df.shape)\n\n    join_df = pdm_df[pdm_df[\'id\'] == id][:params[\'seq_length\'] + current_cycle][pdm_df[pdm_df[\'id\'] == id][:params[\'seq_length\'] + current_cycle].columns.difference(cols_normalize_train)].join(norm_pdm_df)\n    #print("shape of joint_df: ",join_df.shape)\n    pdm_df_eval_online = join_df.reindex(columns=pdm_df[pdm_df[\'id\'] == id][current_cycle:params[\'seq_length\'] + current_cycle].columns)\n    \n    #print("shape of pdm_df_eval_online: 

In [47]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_df[cols_normalize_train])


def prepare_sequence_data(pdm_df,scaler, id, current_cycle, params, cols_normalize_train, sequence_cols):
    
    # In prepare_sequence_data
    norm_pdm_df = pd.DataFrame(scaler.transform(pdm_df[pdm_df['id'] == id][cols_normalize_train][:params['seq_length'] + current_cycle]),
                               columns=cols_normalize_train,
                               index=pdm_df[pdm_df['id'] == id][:params['seq_length'] + current_cycle].index)

    join_df = pdm_df[pdm_df['id'] == id][:params['seq_length'] + current_cycle][pdm_df[pdm_df['id'] == id][:params['seq_length'] + current_cycle].columns.difference(cols_normalize_train)].join(norm_pdm_df)
    #print("shape of joint_df: ",join_df.shape)
    pdm_df_eval_online = join_df.reindex(columns=pdm_df[pdm_df['id'] == id][current_cycle:params['seq_length'] + current_cycle].columns)
    
    #print("shape of pdm_df_eval_online: ",pdm_df_eval_online.shape)
    seq_array_test_k = pdm_df_eval_online[sequence_cols].values[current_cycle:params['seq_length'] + current_cycle]
    #print("shape of seq_array_test_k before reshaping: ",seq_array_test_k.shape)
    seq_array_test_k = np.asarray(seq_array_test_k).astype(np.float32).reshape(1, params['seq_length'], len(sequence_cols))
    #print("shape of seq_array_test_k after reshaping: ",seq_array_test_k.shape)
    
    return seq_array_test_k


In [48]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import torch
from scipy.optimize import minimize_scalar

                                                           
def calculate_probabilities_for_pdm_policy_1_Without_ordering(estimator, pdm_df,scaler,params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT,C_unav, C_inv, device):
    counter = 0
    t_LC_array = np.zeros(len(pdm_df['id'].unique()))
    costs_rep_array = np.zeros(len(pdm_df['id'].unique()))
    costs_delay_array = np.zeros(len(pdm_df['id'].unique()))
    costs_stock_array = np.zeros(len(pdm_df['id'].unique()))
    
    for id in pdm_df['id'].unique():
        preventive_replacement = False
        
        
        for current_cycle in range(pdm_df[pdm_df['id'] == id].shape[0] - params['seq_length'] + 1):
            if current_cycle in array_decisions:
                # Prepare data                           
                seq_array_test_k = prepare_sequence_data(pdm_df,scaler, id, current_cycle, params, cols_normalize_train, sequence_cols)
                
                # Convert to tensor
                seq_tensor = torch.tensor(seq_array_test_k, dtype=torch.float32).to(device)
                
                # Predict
                with torch.no_grad():
                    outputs = estimator(seq_tensor).squeeze()
                probabilities = torch.softmax(outputs, dim=1).cpu().numpy()
                
                #print("probabilities of last sequence: ", probabilities[-1,:])
                #print("shape of probabilities: ", probabilities.shape)
                # Calculate probabilities
                prob_RUL_smaller_w1 = (probabilities[-1, 1] + probabilities[-1, 2])
                prob_RUL_smaller_DT = probabilities[-1, 2]
                #prob_RUL_smaller_w1 = np.mean(probabilities[:, 1] + probabilities[:, 2])
                #prob_RUL_smaller_DT = np.mean(probabilities[:, 2])


                # Apply PdM policy 1
                pthres = C_p / C_c  # Heuristic threshold

                if prob_RUL_smaller_DT < pthres:
                    action = "DN"  # Do Nothing
                else:
                    action = "PR"  # Preventive Replacement

                #print(f"Action taken: {action}")

                if action == "PR":
                    T_R_i =  current_cycle + params['seq_length']
                    #T_R_i =  current_cycle
                    T_F_i = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
                    t_LC_array[counter] = min(T_R_i, T_F_i) # length of life cycle of ith component
                    costs_rep_array[counter] = C_p if T_R_i < T_F_i else C_c
                    #print('Preventive replacement informed at cycle:', t_LC_array[counter])
                    preventive_replacement = True
                    costs_delay_array[counter] = max(0, L - t_LC_array[counter]) * C_unav
                    costs_stock_array[counter] = max(0, t_LC_array[counter] - L) * C_inv
                    break

        if not preventive_replacement:
            t_LC_array[counter] = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
            #print('Component failure at t:', t_LC_array[counter])
            costs_rep_array[counter] = C_c

            
        #print('True failure:', pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1])
        #print('-----------------------------------------')
        counter += 1

    return t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array


# Usage

t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_1_Without_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)


In [49]:
t_LC_array

array([240., 200., 280., 260., 180., 270., 170., 200., 210., 150., 130.,
       340., 150., 250., 280., 330., 190., 150., 180., 190.])

In [50]:
costs_rep_array

array([1000.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.])

In [51]:
costs_delay_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [52]:
costs_stock_array

array([220., 180., 260., 240., 160., 250., 150., 180., 190., 130., 110.,
       320., 130., 230., 260., 310., 170., 130., 160., 170.])

## Implementing PDM Policy 1 with Ordering

In [53]:
import pandas as pd
                   
import numpy as np
from sklearn import preprocessing
import torch

def calculate_probabilities_for_pdm_policy_1_With_ordering(estimator, pdm_df,scaler,params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device):
    counter = 0
    t_order_array = np.zeros(len(pdm_df['id'].unique()))
    t_LC_array = np.zeros(len(pdm_df['id'].unique()))
    costs_rep_array = np.zeros(len(pdm_df['id'].unique()))
    costs_delay_array = np.zeros(len(pdm_df['id'].unique()))
    costs_stock_array = np.zeros(len(pdm_df['id'].unique()))
    w = np.ceil(L / DT) * DT
    
    for id in pdm_df['id'].unique():
        #print('ID:', id)
        preventive_replacement = False
        order = False

        for current_cycle in range(pdm_df[pdm_df['id'] == id].shape[0] - params['seq_length'] + 1):
            #print("current_cycle: ", current_cycle)
            if current_cycle in array_decisions:
                # Prepare data
                
                seq_array_test_k = prepare_sequence_data(pdm_df,scaler, id, current_cycle, params, cols_normalize_train, sequence_cols)
                
                # Convert to tensor
                seq_tensor = torch.tensor(seq_array_test_k, dtype=torch.float32).to(device)

                # Predict
                with torch.no_grad():
                    outputs = estimator(seq_tensor).squeeze()
                probabilities = torch.softmax(outputs, dim=1).cpu().numpy()

                #print("probabilities of last sequence: ", probabilities[-1,:])
                #print("shape of probabilities: ", probabilities.shape)
                # Calculate probabilities
                prob_RUL_smaller_w1 = (probabilities[-1, 1] + probabilities[-1, 2])
                prob_RUL_smaller_DT = probabilities[-1, 2]
                #prob_RUL_smaller_w1 = np.mean(probabilities[:, 1] + probabilities[:, 2])
                #prob_RUL_smaller_DT = np.mean(probabilities[:, 2])


                # Apply PdM policy 1
                p_order_thres = C_p / C_c  # Heuristic threshold for ordering
                p_rep_thres = C_p / C_c  # Heuristic threshold for replacement

                #w = (L // params['seq_length']) * params['seq_length']  # Adjusted lead time

                # Ordering decision
                if not order and prob_RUL_smaller_w1 >= p_order_thres:
                    
                    t_order_array[counter] = params['seq_length'] + current_cycle + w
                    #t_order_array[counter] =  current_cycle + w
                    order = True
                    #print(f"Component ordered at cycle: {t_order_array[counter]}")

                # Apply PdM policy 1
                
                if prob_RUL_smaller_DT < p_rep_thres:
                    action = "DN"  # Do Nothing
                else:
                    action = "PR"  # Preventive Replacement

                #print(f"Action taken: {action}")

                if action == "PR":
                    T_R_i = params['seq_length'] + current_cycle
                    #T_R_i =  current_cycle
                    T_F_i = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
                    t_LC_array[counter] = min(T_R_i, T_F_i)
                    costs_rep_array[counter] = C_p if T_R_i < T_F_i else C_c
                    #print('Preventive replacement informed at cycle:', t_LC_array[counter])

                    preventive_replacement = True
                    costs_delay_array[counter] = max(0, L - t_LC_array[counter]) * C_unav

                    costs_stock_array[counter] = max(0, t_LC_array[counter] - L) * C_inv
                    break

        if not preventive_replacement:
            t_LC_array[counter] = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
            #print('Component failure at t:', t_LC_array[counter])
            costs_rep_array[counter] = C_c

            if not order:
                costs_delay_array[counter] = L * C_unav
                costs_stock_array[counter] = 0  # No stock cost if no order was placed
            else:
                costs_delay_array[counter] = max(t_order_array[counter] + L - t_LC_array[counter], 0) * C_unav
                costs_stock_array[counter] = max(t_LC_array[counter] - (t_order_array[counter] + L), 0) * C_inv

        #print('True failure:', pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1])
        #print('-----------------------------------------')
        counter += 1

    return t_order_array, t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array

                                                                                               
# Usage

t_order_array, t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_1_With_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)


In [54]:
t_order_array

array([230., 210., 290., 260., 180., 270., 160., 200., 200., 150., 130.,
       340., 140., 260., 280., 330., 200., 140., 180., 180.])

In [55]:
t_LC_array

array([240., 200., 280., 260., 180., 270., 170., 200., 210., 150., 130.,
       340., 150., 250., 280., 330., 190., 150., 180., 190.])

In [56]:
costs_rep_array

array([1000.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.])

In [57]:
costs_delay_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [58]:
costs_stock_array

array([220., 180., 260., 240., 160., 250., 150., 180., 190., 130., 110.,
       320., 130., 230., 260., 310., 170., 130., 160., 170.])

## Implementing PDM policy 2

1. `Objective:` PDM policy 2 aims to find the optimal time for preventive replacement (T_{R,k}*) by minimizing the long-run expected maintenance cost per unit time. <br>
2. `Decision Rule:` At each time step t_k, the policy decides:
> * Perform Preventive Replacement (PR) if t_k + ΔT ≥ T_{R,k}*
> * Do Nothing (DN) otherwise
3. `Optimization Problem:` The policy solves an optimization problem at each time step to find T_{R,k}* by minimizing the objective function f(T_{R,k}) given in Equation 12. <br>
4. `Components of the Objective Function (Equation 12):`
> $f\left(T_{\mathrm{R}, k}\right)=\frac{\mathrm{E}\left[C_{\mathrm{rep}}\left(T_{\mathrm{R}, k}\right)\right]}{\mathrm{E}\left[T_{\mathrm{lc}}\left(T_{\mathrm{R}, k}\right)\right]}=\frac{P_{\mathrm{PR}} \cdot c_{\mathrm{p}}+\left(1-P_{\mathrm{PR}}\right) \cdot c_{\mathrm{c}}}{P_{\mathrm{PR}} \cdot\left(T_{\mathrm{R}, k}\right)+\int_t^{T_{\mathrm{R}, k}} t f_{R U L_{\mathrm{Pred}, k}}\left(t-t_k\right) \mathrm{d} t}$
> * E[C_{rep}(T_{R,k})]: Expected cost of replacement
> * E[T_{lc}(T_{R,k})]: Expected lifecycle time
> * P_{PR}: Probability of preventive replacement (defined in Equation 13)
> * c_p: Cost of preventive replacement
> * c_c: Cost of corrective replacement (failure)
> * f_{RUL_{pred,k}}(t): Full distribution of the RUL prediction at time t_k
> * The find_optimal_replacement_time function implements the objective function from PDM policy 2. It finds the optimal replacement time T_R_k by minimizing the expected cost per unit time. <br>

5. Interpretation of Equation 12:
> * Numerator: Represents the expected cost, considering both preventive and corrective replacements.
> * Denominator: Represents the expected lifecycle time.
> * By minimizing this ratio, the policy aims to find the optimal balance between cost and component lifetime.


6. Probability of Preventive Replacement(Equation 13): 
> P_{PR} represents the probability that the component will be preventively replaced at T_{R,k}.
> It's calculated by integrating the RUL prediction distribution from T_{R,k} to infinity.

> $$
P_{\mathrm{PR}}=\int_{T_{\mathrm{R}, k}}^{\infty} f_{R U L_{\mathrm{Pred}, k}}\left(t-t_k\right) \mathrm{d} t
$$       

7. Full RUL Distribution: Unlike PDM policy 1, this policy uses the full distribution of the Remaining Useful Life (RUL) prediction, allowing for more nuanced decision-making.<br>

8. The function `find_optimal_replacement_time` and its Parameters: function is designed to determine the optimal time for preventive replacement in a Predictive Maintenance (PdM) system. It uses a lognormal distribution fitted to the predicted probabilities of component failure to calculate the expected cost per unit time and find the optimal replacement time.<br>
 , representing the probability of needing a preventive replacement after T_R_K. <br>
> `probabilities:` The predicted probabilities of class1 and class2 from the model. <br>
> C_p, C_c, current_cycle, seq_length <br>
> `Fitting the Lognormal Distribution:` `mu, sigma = fit_lognormal_cdf(probabilities):` This fits a lognormal distribution to the given probabilities, returning the parameters mu and sigma..
> * `Probability of Preventive Replacement (P_PR):` This is the probability that the component survives until T_R_k. <br>
> * `Expected Replacement Cost (E_C_rep) :` It's a weighted sum of preventive (C_p) and corrective (C_c) replacement costs.<br> 
> * `Expected Lifecycle Time (E_T_lc) :` The first term (P_PR * T_R_k) is the expected time if preventive replacement occurs
and The sum calculates the expected time if failure occurs before T_R_k.<br>

9. `Optimization:`
> * Uses scipy's `minimize_scalar` function to find the T_R_k that minimizes the objective function.<br>
> * The search is bounded between the current cycle and the end of the sequence.<br>


## Without ordering

In [89]:
from scipy.optimize import minimize
from scipy.stats import lognorm
from scipy.optimize import minimize_scalar
from scipy.integrate import quad
from scipy import stats


def calculate_probabilities_for_pdm_policy_2_Without_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT,C_unav, C_inv, device):
    counter = 0
    t_LC_array = np.zeros(len(pdm_df['id'].unique()))
    costs_rep_array = np.zeros(len(pdm_df['id'].unique()))
    costs_delay_array = np.zeros(len(pdm_df['id'].unique()))
    costs_stock_array = np.zeros(len(pdm_df['id'].unique()))

    for id in pdm_df['id'].unique():
        
        preventive_replacement = False
        for current_cycle in range(params['seq_length'], pdm_df[pdm_df['id'] == id].shape[0] + 1):
            
            if current_cycle in array_decisions:
                #print('current_cycle: ', current_cycle)
                # Prepare data
                seq_array_test_k = prepare_sequence_data(pdm_df,scaler, id, current_cycle, params, cols_normalize_train, sequence_cols)
                
                # Convert to tensor
                seq_tensor = torch.tensor(seq_array_test_k, dtype=torch.float32).to(device)

                # Predict
                with torch.no_grad():
                    outputs = estimator(seq_tensor).squeeze()
                probabilities = torch.softmax(outputs, dim=1).cpu().numpy()
                
                #print("probabilities: ", probabilities, "shape of probability: ", probabilities.shape)
                # Calculate optimal replacement time
                T_R_k_optimal = find_optimal_replacement_time_for_PDM_Policy2(probabilities, C_p, C_c, current_cycle,DT, params)
                
                #sequence_end = current_cycle + params['seq_length']
                #print("current_cycle: ", current_cycle,"|| DT: ", DT, "||T_R_k_optimal: ", T_R_k_optimal)

                # Apply PDM policy 2
                if DT >= T_R_k_optimal:
                #if sequence_end + DT >= T_R_k_optimal:
                    action = "PR"  # Preventive Replacement
                else:
                    action = "DN"  # Do Nothing


                # Evaluate decision heuristics

                # Preventive replacement decision
                if action == "PR":
                    #T_R_i = params['seq_length'] + current_cycle
                    T_R_i =  current_cycle
                    print("T_R_i: ", T_R_i)
                    #T_R_i =  T_R_k_optimal
                    T_F_i = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
                    print("T_F_i: ", T_F_i)
                    t_LC_array[counter] = min(T_R_i, T_F_i)
                    costs_rep_array[counter] = C_p if T_R_i < T_F_i else C_c
                    #print('Preventive replacement informed at cycle:', t_LC_array[counter])
                    preventive_replacement = True
                    costs_delay_array[counter] = max(0, L - t_LC_array[counter]) * C_unav
                    costs_stock_array[counter] = max(0, t_LC_array[counter] - L) * C_inv
                    break

        if not preventive_replacement:
            t_LC_array[counter] = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
            #print('Component failure at t:', t_LC_array[counter])
            costs_rep_array[counter] = C_c

            
        #print('True failure:', pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1])
        #print('-----------------------------------------')
        counter += 1

    return t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array




def expected_time_to_failure(T_R_k, current_cycle, mu, sigma):
    """Calculate the expected time to failure."""
    # Define the integrand for E[T_lc]
    def integrand(t):
        return t * lognorm.pdf(t - current_cycle, mu, sigma)

    #Calculate the integral from current_cycle + params['seq_length'] to T_R_k
    integral_value, error = quad(integrand, current_cycle , T_R_k, limit=200)
        
    return integral_value



def lognorm_cdf(x, mu, sigma):
    return lognorm.cdf(x, s=sigma, scale=np.exp(mu))


def fit_lognormal_cdf(probabilities):
    # Extract probabilities for class 1 and 2 from the last sequence
    p1 = probabilities[-1, 1]  # Probability of class 1
    p2 = probabilities[-1, 2]  # Probability of class 2

    # Estimate parameters of lognormal distribution
    # We'll use two points: (1, p1) and (2, p1+p2)
    x = np.array([1, 2])
    y = np.array([p1, p1+p2])

    # Use scipy.stats to fit the lognormal CDF
    shape, loc, scale = stats.lognorm.fit(x, y, floc=0)
    
    # Convert lognorm parameters to mu and sigma
    sigma = shape
    mu = np.log(scale)
    
    return mu, sigma




def find_optimal_replacement_time_for_PDM_Policy2(probabilities, C_p, C_c, current_cycle,DT, params):
    """Find the optimal replacement time for PdM policy 2."""
    #sequence_end = current_cycle + params['seq_length']
    mu, sigma = fit_lognormal_cdf(probabilities)
    
    def objective_function(T_R_k):
        # P_PR: Probability that the component will survive beyond time T_R_k
        #P_PR = 1 - lognorm.cdf(T_R_k - sequence_end, s=sigma, scale=np.exp(mu))
        P_PR = 1 - lognorm.cdf(T_R_k - current_cycle, s=sigma, scale=np.exp(mu))
        
        # Calculate expected cost of replacement
        E_C_rep = P_PR * C_p + (1 - P_PR) * C_c
        
        # Calculate expected time to failure using the fitted lognormal distribution
        #E_T_lc = P_PR * T_R_k + expected_time_to_failure(T_R_k, sequence_end, mu, sigma)
        E_T_lc = P_PR * T_R_k + expected_time_to_failure(T_R_k, current_cycle, mu, sigma)
        
        return E_C_rep / E_T_lc

    # Use minimize_scalar to find the optimal replacement time
    #result = minimize_scalar(objective_function, bounds=(sequence_end, sequence_end + DT), method='bounded')
    result = minimize_scalar(objective_function, bounds=(0, 2*DT), method='bounded')
    
    return result.x
# Usage
t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_2_Without_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)


current_cycle:  0
current_cycle:  0 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  10
current_cycle:  10 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  20
current_cycle:  20 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  30
current_cycle:  30 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  40
current_cycle:  40 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  50
current_cycle:  50 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  60
current_cycle:  60 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  70
current_cycle:  70 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  80
current_cycle:  80 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  90
current_cycle:  90 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  100
current_cycle:  100 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  110
current_cycle:  110 || DT:  10 ||T_R_k_optima

current_cycle:  130 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  0
current_cycle:  0 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  10
current_cycle:  10 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  20
current_cycle:  20 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  30
current_cycle:  30 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  40
current_cycle:  40 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  50
current_cycle:  50 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  60
current_cycle:  60 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  70
current_cycle:  70 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  80
current_cycle:  80 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  90
current_cycle:  90 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  100
current_cycle:  100 || DT:  10 ||T_R_k_optimal:  19.9999949332021

current_cycle:  50 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  60
current_cycle:  60 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  70
current_cycle:  70 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  80
current_cycle:  80 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  90
current_cycle:  90 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  100
current_cycle:  100 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  110
current_cycle:  110 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  120
current_cycle:  120 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  130
current_cycle:  130 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  140
current_cycle:  140 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  150
current_cycle:  150 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  160
current_cycle:  160 || DT:  10 ||T_R_k_optimal:  19.

current_cycle:  180 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  190
current_cycle:  190 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  200
current_cycle:  200 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  210
current_cycle:  210 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  220
current_cycle:  220 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  230
current_cycle:  230 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  240
current_cycle:  240 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  250
current_cycle:  250 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  260
current_cycle:  260 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  270
current_cycle:  270 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  280
current_cycle:  280 || DT:  10 ||T_R_k_optimal:  19.99999493320217
current_cycle:  0
current_cycle:  0 || DT:  10 ||T_R_k_optimal:

In [61]:
t_LC_array

array([240., 214., 293., 267., 188., 278., 178., 213., 217., 154., 135.,
       341., 155., 258., 283., 336., 202., 156., 185., 200.])

In [62]:
costs_rep_array

array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000., 1000.])

In [63]:
costs_delay_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [64]:
costs_stock_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

## with ordering

In [65]:
import numpy as np
import torch
from scipy.optimize import minimize_scalar
from scipy.stats import lognorm

def calculate_probabilities_for_pdm_policy_2_With_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device):
    counter = 0
    t_order_array = np.zeros(len(pdm_df['id'].unique()))
    t_LC_array = np.zeros(len(pdm_df['id'].unique()))
    costs_rep_array = np.zeros(len(pdm_df['id'].unique()))
    costs_delay_array = np.zeros(len(pdm_df['id'].unique()))
    costs_stock_array = np.zeros(len(pdm_df['id'].unique()))
    w = np.ceil(L / DT) * DT
    #ΔT = params['seq_length']  # Time interval between decision points
    #w = (L // ΔT) * ΔT  # Adjusted lead time

    for id in pdm_df['id'].unique():
        #print('ID:', id)
        preventive_replacement = False
        order = False

        for current_cycle in range(pdm_df[pdm_df['id'] == id].shape[0] - params['seq_length'] + 1):
            #print("current_cycle: ", current_cycle)
            if current_cycle in array_decisions:
                # Prepare data
                seq_array_test_k = prepare_sequence_data(pdm_df,scaler, id, current_cycle, params, cols_normalize_train, sequence_cols)
                # Convert to tensor
                seq_tensor = torch.tensor(seq_array_test_k, dtype=torch.float32).to(device)

                # Predict
                with torch.no_grad():
                    outputs = estimator(seq_tensor).squeeze()
                probabilities = torch.softmax(outputs, dim=1).cpu().numpy()

                # Calculate optimal replacement time
                T_R_k_optimal = find_optimal_replacement_time_for_PDM_Policy2(probabilities, C_p, C_c, current_cycle, DT, params)
                #print("current_cycle: ", current_cycle,"||current_cycle + params['seq_length']: ", current_cycle + params['seq_length'], "||T_R_k_optimal: ", T_R_k_optimal)
                # Apply PDM policy 2
                if DT >= T_R_k_optimal:
                #if sequence_end + DT >= T_R_k_optimal:
                    action = "PR"  # Preventive Replacement
                else:
                    action = "DN"  # Do Nothing

                #print(f"Action taken: {action}")

                # Ordering decision
                #prob_RUL_smaller_w1 = np.mean(probabilities[:, 1] + probabilities[:, 2])
                prob_RUL_smaller_w1 = probabilities[-1, 1] + probabilities[-1, 2]
                p_order_thres = C_p / C_c  # Heuristic threshold for ordering
                if not order and prob_RUL_smaller_w1 >= p_order_thres:
                    t_order_array[counter] = params['seq_length'] + current_cycle + w
                    
                    order = True
                    #print(f"Component ordered at cycle: {t_order_array[counter]}")

                # Preventive replacement decision
                if action == "PR":
                    T_R_i = params['seq_length'] + current_cycle
                    #T_R_i =  T_R_k_optimal
                    T_F_i = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
                    t_LC_array[counter] = min(T_R_i, T_F_i)
                    costs_rep_array[counter] = C_p if T_R_i < T_F_i else C_c
                    #print('Preventive replacement informed at cycle:', t_LC_array[counter])
                    preventive_replacement = True
                    costs_delay_array[counter] = max(0, L - t_LC_array[counter]) * C_unav
                    costs_stock_array[counter] = max(0, t_LC_array[counter] - L) * C_inv
                    break

        if not preventive_replacement:
            t_LC_array[counter] = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
            #print('Component failure at t:', t_LC_array[counter])
            costs_rep_array[counter] = C_c

            if not order:
                costs_delay_array[counter] = L * C_unav
                costs_stock_array[counter] = 0  # No stock cost if no order was placed
            else:
                costs_delay_array[counter] = max(t_order_array[counter] + L - t_LC_array[counter], 0) * C_unav
                costs_stock_array[counter] = max(t_LC_array[counter] - (t_order_array[counter] + L), 0) * C_inv

        #print('True failure:', pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1])
        #print('-----------------------------------------')
        counter += 1

    return t_order_array, t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array


def expected_time_to_failure(T_R_k, current_cycle, mu, sigma):
    """Calculate the expected time to failure."""
    # Define the integrand for E[T_lc]
    def integrand(t):
        return t * lognorm.pdf(t - current_cycle, mu, sigma)

    #Calculate the integral from current_cycle + params['seq_length'] to T_R_k
    integral_value, error = quad(integrand, current_cycle , T_R_k, limit=200)
    
    return integral_value



def lognorm_cdf(x, mu, sigma):
    return lognorm.cdf(x, s=sigma, scale=np.exp(mu))


def fit_lognormal_cdf(probabilities):
    # Extract probabilities for class 1 and 2 from the last sequence
    p1 = probabilities[-1, 1]  # Probability of class 1
    p2 = probabilities[-1, 2]  # Probability of class 2

    # Estimate parameters of lognormal distribution
    # We'll use two points: (1, p1) and (2, p1+p2)
    x = np.array([1, 2])
    y = np.array([p1, p1+p2])

    # Use scipy.stats to fit the lognormal CDF
    shape, loc, scale = stats.lognorm.fit(x, y, floc=0)
    
    # Convert lognorm parameters to mu and sigma
    sigma = shape  
    mu = np.log(scale) 
    
    return mu, sigma



def find_optimal_replacement_time_for_PDM_Policy2(probabilities, C_p, C_c, current_cycle,DT, params):
    """Find the optimal replacement time for PdM policy 2."""
    sequence_end = current_cycle + params['seq_length']
    mu, sigma = fit_lognormal_cdf(probabilities)
    
    def objective_function(T_R_k):
        # P_PR: Probability that the component will survive beyond time T_R_k
        P_PR = 1 - lognorm.cdf(T_R_k - sequence_end, s=sigma, scale=np.exp(mu))
        
        # Calculate expected cost of replacement
        E_C_rep = P_PR * C_p + (1 - P_PR) * C_c
        
        # Calculate expected time to failure using the fitted lognormal distribution
        E_T_lc = P_PR * T_R_k + expected_time_to_failure(T_R_k, sequence_end, mu, sigma)
        
        return E_C_rep / E_T_lc

    # Use minimize_scalar to find the optimal replacement time
    #result = minimize_scalar(objective_function, bounds=(sequence_end, sequence_end + DT), method='bounded')
    result = minimize_scalar(objective_function, bounds=(0, 2*DT), method='bounded')
    
    return result.x

# Usage
t_order_array, t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_2_With_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)


In [66]:
t_order_array

array([230., 210., 290., 260., 180., 270., 160., 200., 200., 150., 130.,
       340., 140., 260., 280., 330., 200., 140., 180., 180.])

In [67]:
t_LC_array

array([240., 214., 293., 267., 188., 278., 178., 213., 217., 154., 135.,
       341., 155., 258., 283., 336., 202., 156., 185., 200.])

In [68]:
costs_rep_array

array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000., 1000.])

In [69]:
costs_delay_array

array([100., 160., 170., 130., 120., 120.,  20.,  70.,  30., 160., 150.,
       190.,  50., 220., 170., 140., 180.,  40., 150.,   0.])

In [70]:
costs_stock_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

## PDM Policy 3 

1. `Objective: `PDM Policy 3 represents a significant advancement in predictive maintenance by utilizing the full distribution of RUL data to inform maintenance decisions. The objective function captures both the expected costs of preventive and corrective actions and the additional costs associated with early replacements, providing a comprehensive framework for optimizing maintenance strategies. <br>
2. `Explanation of Equation(14): ` The equation for the objective function in PDM Policy 3 is given as:
> $f\left(T_{\mathrm{R}, k}\right)=P_{\mathrm{PR}} \cdot C_{\mathrm{p}}+\left(1-P_{\mathrm{PR}}\right) \cdot C_{\mathrm{c}}+\int_{T_{\mathrm{R}, k}}^{\infty}\left(t-T_{\mathrm{R}, k}\right) \cdot \frac{\mathrm{E}_{\bar{T}_{\mathrm{F}}}\left[C_{\mathrm{rep}}\right]}{\mathrm{E}_{\bar{T}_{\mathrm{F}}}\left[T_{l \mathrm{c}}\right]} f_{R U L_{\mathrm{pred}, k}}\left(t-t_k\right) \mathrm{d} t$

3. `Components of the equation: `

>  1. `Expected Replacement Cost: `
>> * The first two terms, $P_{\mathrm{PR}} \cdot C_{\mathrm{p}}$ and $\left(1-P_{\mathrm{PR}}\right) \cdot C_{\mathrm{c}}$ represent the expected costs associated with preventive and corrective replacements, respectively. <br>
>> * $P_{\mathrm{PR}}$ is the probability that the components will survive until the replacement time $T_{\mathrm{R}, k}$ <br>
>> * $ C_{\mathrm{p}}$ is the cost of preventive replacement, and $ C_{\mathrm{c}}$ is the cost of corrective replacement. <br>

> 2. `Integral Term: `
>> * The integral term quantifies the additional expected maintenance cost associated with an "early" replacement at $T_{\mathrm{R}, k}$ <br>
>> * The expression $ \left(t-T_{\mathrm{R}, k}\right) $ represents the time lost due to an early replacement.
>> * The term $ \frac{\mathrm{E}_{\bar{T}_{\mathrm{F}}}\left[C_{\mathrm{rep}}\right]}{\mathrm{E}_{\bar{T}_{\mathrm{F}}}\left[T_{l \mathrm{c}}\right]} $  is the long-run expected maintenance cost per unit time concerning the distribution of the population of components. It provides a scaling factor for the additional cost incurred by replacing the component early. <br> 
>> * $ f_{R U L_{\mathrm{pred}, k}}\left(t-t_k\right) $is the predicted probability density function of the RUL, representing the likelihood of failure occurring at time t after the current cycle $t_k$

## without ordering

In [71]:
import numpy as np
from scipy.integrate import quad, simps, romberg
from scipy.stats import lognorm
from scipy import optimize
from scipy.optimize import minimize_scalar

#def calculate_probabilities_for_pdm_policy_3_Without_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device):
def calculate_probabilities_for_pdm_policy_3_Without_ordering(estimator, pdm_df,scaler, train_df, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device):
    counter = 0
    t_LC_array = np.zeros(len(pdm_df['id'].unique()))
    costs_rep_array = np.zeros(len(pdm_df['id'].unique())) 
    costs_delay_array = np.zeros(len(pdm_df['id'].unique()))
    costs_stock_array = np.zeros(len(pdm_df['id'].unique()))

    for id in pdm_df['id'].unique():
        #print('ID:', id)
        preventive_replacement = False

        for current_cycle in range(pdm_df[pdm_df['id'] == id].shape[0] - params['seq_length'] + 1):      
            if current_cycle in array_decisions:
                # Prepare data
                seq_array_test_k = prepare_sequence_data(pdm_df,scaler, id, current_cycle, params, cols_normalize_train, sequence_cols)
                # Convert to tensor
                seq_tensor = torch.tensor(seq_array_test_k, dtype=torch.float32).to(device)

                # Predict
                with torch.no_grad():
                    outputs = estimator(seq_tensor).squeeze()
                probabilities = torch.softmax(outputs, dim=1).cpu().numpy()




                # Calculate optimal replacement time
                T_R_k_optimal = find_optimal_replacement_time_for_PDM_Policy3(probabilities, C_p, C_c, train_df, current_cycle,DT, params)

                #print("current_cycle: ", current_cycle,"||current_cycle + DT: ", current_cycle + DT, "||T_R_k_optimal: ", T_R_k_optimal)
                sequence_end = current_cycle + params['seq_length']
                # Apply PDM policy 2 but calculate T_R_k_optimal using PDM Policy 3
                if DT >= T_R_k_optimal:
                #if sequence_end + DT >= T_R_k_optimal:
                    action = "PR"  # Preventive Replacement
                else:
                    action = "DN"  # Do Nothing
                #print(f"Action taken: {action}")

                # Evaluate decision heuristics

                # Preventive replacement decision
                if action == "PR":
                    T_R_i = params['seq_length'] + current_cycle
                    #T_R_i =  int(T_R_k_optimal)
                    T_F_i = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
                    t_LC_array[counter] = min(T_R_i, T_F_i)
                    costs_rep_array[counter] = C_p if T_R_i < T_F_i else C_c
                    #print('Preventive replacement informed at cycle:', t_LC_array[counter])
                    preventive_replacement = True
                    costs_delay_array[counter] = max(0, L - t_LC_array[counter]) * C_unav
                    costs_stock_array[counter] = max(0, t_LC_array[counter] - L) * C_inv
                    break

        if not preventive_replacement:
            t_LC_array[counter] = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
            #print('Component failure at t:', t_LC_array[counter])
            costs_rep_array[counter] = C_c

            
        #print('True failure:', pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1])
        #print('-----------------------------------------')
        counter += 1
        

    return t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array



#def lognorm_cdf(x, mu, sigma):
#    return lognorm.cdf(x, s=sigma, scale=np.exp(mu))


def fit_lognormal_cdf(probabilities):
    # Extract probabilities for class 1 and 2 from the last sequence
    p1 = probabilities[-1, 1]  # Probability of class 1
    p2 = probabilities[-1, 2]  # Probability of class 2

    # Estimate parameters of lognormal distribution
    # We'll use two points: (1, p1) and (2, p1+p2)
    x = np.array([1, 2])
    y = np.array([p1, p1+p2])

    # Use scipy.stats to fit the lognormal CDF
    shape, loc, scale = stats.lognorm.fit(x, y, floc=0)
    
    # Convert lognorm parameters to mu and sigma
    sigma = shape
    mu = np.log(scale)
    
    return mu, sigma

"""
def fit_lognormal_cdf(probabilities):
    # Extract probabilities for class 1 and 2 from the last sequence
    p1 = probabilities[-1, 1]  # Probability of class 1
    p2 = probabilities[-1, 2]  # Probability of class 2

    # Estimate parameters of lognormal distribution
    # We'll use two points: (1, p1) and (2, p1+p2)
    x = np.array([1, 2])
    y = np.array([p1, p1+p2])

    # Use scipy.stats to fit the lognormal CDF
    shape, loc, scale = stats.lognorm.fit(x, y, floc=0)
    
    # Convert lognorm parameters to mu and sigma
    #sigma = shape
    #mu = np.log(scale)
    
    return shape, loc, scale

"""

def calculate_expected_values(train_df):
    mu_TF = 0
    unique_ids = train_df['id'].unique()
    for id in unique_ids:
        mu_TF += train_df[train_df['id'] == id]['cycle'].iloc[-1]
    return  mu_TF / len(unique_ids)


def find_optimal_replacement_time_for_PDM_Policy3(probabilities, C_p, C_c, train_df, current_cycle, DT, params):
    sequence_end = current_cycle + params['seq_length']
    mu, sigma = fit_lognormal_cdf(probabilities)
    #shape, loc, scale = fit_lognormal_cdf(probabilities)
    mu_TF = calculate_expected_values(train_df)
    #print('mu_TF: ',mu_TF)
    #mu_TF = calculate_expected_values(pdm_df)
    
    def objective_function(T_R_k):
        P_PR = 1 - lognorm.cdf(T_R_k - sequence_end, mu, sigma)
        #P_PR = 1 - lognorm.cdf(T_R_k - sequence_end, s=sigma, scale=np.exp(mu))
        #P_PR = 1 - lognorm.cdf(T_R_k - sequence_end, shape, loc, scale)
        
        def integral_term(t):
            return (t - T_R_k) * lognorm.pdf(t - sequence_end, mu, sigma)
            #return (t - T_R_k) * lognorm.pdf(t - sequence_end, s=sigma, scale=np.exp(mu))
            #return (t - T_R_k) * lognorm.pdf(t - sequence_end, shape, loc, scale)
        additional_cost, _ = quad(integral_term, T_R_k, np.inf, limit=200)
        additional_cost *= C_p / mu_TF
        
        return P_PR * C_p + (1 - P_PR) * C_c + additional_cost
    
    # Find the optimal replacement time using bounded optimization
    #result = minimize_scalar(objective_function, bounds=(sequence_end, sequence_end + 2*DT), method='bounded')
    result = minimize_scalar(objective_function, bounds=(0,2*DT), method='bounded')
    T_R_k_optimal = result.x
    
    return T_R_k_optimal

# Usage                                                                                                                               
t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_3_Without_ordering(estimator, pdm_df,scaler,train_df, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)
#t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_3_Without_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)


In [72]:
t_LC_array

array([80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80.,
       80., 80., 80., 80., 80., 80., 80.])

In [73]:
costs_rep_array

array([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100., 100., 100., 100., 100., 100., 100., 100., 100.])

In [74]:
costs_delay_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [75]:
costs_stock_array

array([60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60.,
       60., 60., 60., 60., 60., 60., 60.])

## With Ordering

In [76]:
def calculate_probabilities_for_pdm_policy_3_With_ordering(estimator, pdm_df,scaler,train_df, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device):
    counter = 0
    t_order_array = np.zeros(len(pdm_df['id'].unique()))
    t_LC_array = np.zeros(len(pdm_df['id'].unique()))
    costs_rep_array = np.zeros(len(pdm_df['id'].unique()))
    costs_delay_array = np.zeros(len(pdm_df['id'].unique()))
    costs_stock_array = np.zeros(len(pdm_df['id'].unique()))
    w = np.ceil(L / DT) * DT
    #ΔT = params['seq_length']  # Time interval between decision points
    #w = (L // ΔT) * ΔT  # Adjusted lead time

    for id in pdm_df['id'].unique():
        #print('ID:', id)
        preventive_replacement = False
        order = False

        for current_cycle in range(pdm_df[pdm_df['id'] == id].shape[0] - params['seq_length'] + 1):
            if current_cycle in array_decisions:
                # Prepare data
                seq_array_test_k = prepare_sequence_data(pdm_df,scaler, id, current_cycle, params, cols_normalize_train, sequence_cols)
                
                # Convert to tensor
                seq_tensor = torch.tensor(seq_array_test_k, dtype=torch.float32).to(device)

                # Predict
                with torch.no_grad():
                    outputs = estimator(seq_tensor).squeeze()
                probabilities = torch.softmax(outputs, dim=1).cpu().numpy()



                # Calculate optimal replacement time
                T_R_k_optimal = find_optimal_replacement_time_for_PDM_Policy3(probabilities, C_p, C_c, train_df, current_cycle,DT, params)
                #print("current_cycle: ", current_cycle,"||current_cycle + DT : ", current_cycle + DT, "||T_R_k_optimal: ", T_R_k_optimal)
                
                # Apply PDM policy 2 but calculate T_R_k_optimal using PDM Policy 3
                if DT >= T_R_k_optimal:
                #if sequence_end + DT >= T_R_k_optimal:
                    action = "PR"  # Preventive Replacement
                else:
                    action = "DN"  # Do Nothing

                # Ordering decision
                #prob_RUL_smaller_w1 = np.mean(probabilities[:, 1] + probabilities[:, 2])
                prob_RUL_smaller_w1 = probabilities[-1, 1] + probabilities[-1, 2]
                p_order_thres = C_p / C_c  # Heuristic threshold for ordering
                if not order and prob_RUL_smaller_w1 >= p_order_thres:
                    t_order_array[counter] = params['seq_length'] + current_cycle+ w
                    #t_order_array[counter] = current_cycle+ w
                    order = True
                    #print(f"Component ordered at cycle: {t_order_array[counter]}")

                # Preventive replacement decision
                if action == "PR":
                    T_R_i = params['seq_length'] + current_cycle
                    #T_R_i =  T_R_k_optimal
                    T_F_i = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
                    t_LC_array[counter] = min(T_R_i, T_F_i)
                    costs_rep_array[counter] = C_p if T_R_i < T_F_i else C_c
                    #print('Preventive replacement informed at cycle:', t_LC_array[counter])
                    preventive_replacement = True
                    costs_delay_array[counter] = max(0, L - t_LC_array[counter]) * C_unav
                    costs_stock_array[counter] = max(0, t_LC_array[counter] - L) * C_inv
                    break

        if not preventive_replacement:
            t_LC_array[counter] = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
            #print('Component failure at t:', t_LC_array[counter])
            costs_rep_array[counter] = C_c

            if not order:
                costs_delay_array[counter] = L * C_unav
                costs_stock_array[counter] = 0  # No stock cost if no order was placed
            else:
                costs_delay_array[counter] = max(t_order_array[counter] + L - t_LC_array[counter], 0) * C_unav
                costs_stock_array[counter] = max(t_LC_array[counter] - (t_order_array[counter] + L), 0) * C_inv

        #print('True failure:', pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1])
        #print('-----------------------------------------')
        counter += 1

    return t_order_array, t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array


# Usage
t_order_array, t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_3_With_ordering(estimator, pdm_df, scaler,train_df, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)



In [77]:
t_order_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [78]:
t_LC_array

array([80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80.,
       80., 80., 80., 80., 80., 80., 80.])

In [79]:
costs_rep_array

array([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100., 100., 100., 100., 100., 100., 100., 100., 100.])

In [80]:
costs_delay_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [81]:
costs_stock_array

array([60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60.,
       60., 60., 60., 60., 60., 60., 60.])

In [92]:
# Example usage
def calculate_T_R_perfect(pdm_df, id, DT=10):
    # Retrieve the perfect failure time for the ith component
    T_F_perfect = pdm_df[pdm_df['id'] == id]['cycle'].iloc[-1]
    
    # Calculate k as the largest integer such that k * DT < T_F_perfect
    k = int(T_F_perfect // DT)  # Use floor division
    T_R_perfect = k * DT  # Calculate T_R_perfect
    
    return T_R_perfect

def calculate_decision_metric_with_ordering(costs_rep_array, costs_delay_array, costs_stock_array, t_LC_array, C_p, pdm_df):
    #n = len(costs_rep_array)  # Number of components

    # Calculate the average costs and lifecycle times
    average_costs = (np.mean(costs_rep_array) + np.mean(costs_delay_array) + np.mean(costs_stock_array))
    average_t_LC_array = np.mean(t_LC_array)

    # Calculate T_R_perfect for each component and then find the average
    T_R_perfect_array = []
    for id in pdm_df['id'].unique():
        T_R_perfect = calculate_T_R_perfect(pdm_df, id, DT=10)
        T_R_perfect_array.append(T_R_perfect)

    average_T_R_perfect = np.mean(T_R_perfect_array)

    # Calculate the first part of the numerator
    numerator_part1 = average_costs / average_t_LC_array

    # Calculate the second part of the numerator
    numerator_part2 = C_p / average_T_R_perfect

    # Calculate the full numerator
    numerator = numerator_part1 - numerator_part2

    # Calculate the denominator
    denominator = C_p / average_T_R_perfect

    # Calculate the decision-oriented metric
    M_hat = numerator / denominator if denominator != 0 else 0  # Avoid division by zero

    return M_hat

def calculate_decision_metric_without_ordering(costs_rep_array,  t_LC_array, C_p, pdm_df):
    #n = len(costs_rep_array)  # Number of components

    # Calculate the average costs and lifecycle times
    average_costs = np.mean(costs_rep_array)
    average_t_LC_array = np.mean(t_LC_array)

    # Calculate T_R_perfect for each component and then find the average
    T_R_perfect_array = []
    for id in pdm_df['id'].unique():
        T_R_perfect = calculate_T_R_perfect(pdm_df, id, DT=10)
        T_R_perfect_array.append(T_R_perfect)

    average_T_R_perfect = np.mean(T_R_perfect_array)

    # Calculate the first part of the numerator
    numerator_part1 = average_costs / average_t_LC_array

    # Calculate the second part of the numerator
    numerator_part2 = C_p / average_T_R_perfect

    # Calculate the full numerator
    numerator = numerator_part1 - numerator_part2

    # Calculate the denominator
    denominator = C_p / average_T_R_perfect

    # Calculate the decision-oriented metric
    M_hat = numerator / denominator if denominator != 0 else 0  # Avoid division by zero

    return M_hat


# Range of C_p values
C_p_values = np.array([0,100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])
C_c = 1000  # Constant corrective replacement cost
M_hat_Policy1_With_ordering = []
M_hat_Policy1_Without_ordering = []

M_hat_Policy2_With_ordering = []
M_hat_Policy2_Without_ordering = []

M_hat_Policy3_With_ordering = []
M_hat_Policy3_Without_ordering = []
# Calculate M_hat for each C_p
for C_p in C_p_values:
    #Call the function to get updated arrays for policy1 with ordering  
    t_order_array, t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_1_With_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)

    # Calculate M_hat using the updated arrays
    M_hat1 = calculate_decision_metric_with_ordering(costs_rep_array, costs_delay_array, costs_stock_array, t_LC_array, C_p, pdm_df)
    M_hat_Policy1_With_ordering.append(M_hat1)
    
    
    # Call the function to get updated arrays for policy1 without ordering
    t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_1_Without_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)

    # Calculate M_hat using the updated arrays
    M_hat2 = calculate_decision_metric_without_ordering(costs_rep_array,  t_LC_array, C_p, pdm_df)
    M_hat_Policy1_Without_ordering.append(M_hat2)
    
    
    
    # Call the function to get updated arrays for policy2 with ordering
    t_order_array, t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_2_With_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)
    
    # Calculate M_hat using the updated arrays
    M_hat3 = calculate_decision_metric_with_ordering(costs_rep_array, costs_delay_array, costs_stock_array, t_LC_array, C_p, pdm_df)
    M_hat_Policy2_With_ordering.append(M_hat3)
    
    
    # Call the function to get updated arrays for policy2 without ordering
    t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_2_Without_ordering(estimator, pdm_df,scaler, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)
    # Calculate M_hat using the updated arrays
    M_hat4 = calculate_decision_metric_without_ordering(costs_rep_array,  t_LC_array, C_p, pdm_df)
    M_hat_Policy2_Without_ordering.append(M_hat4)
    
    
    # Call the function to get updated arrays for policy3 with ordering                calculate_probabilities_for_pdm_policy_3_With_ordering(estimator, pdm_df, scaler,train_df, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)   
    t_order_array, t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_3_With_ordering(estimator, pdm_df,scaler,train_df, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)
    
    # Calculate M_hat using the updated arrays
    M_hat5 = calculate_decision_metric_with_ordering(costs_rep_array, costs_delay_array, costs_stock_array, t_LC_array, C_p, pdm_df)
    M_hat_Policy3_With_ordering.append(M_hat5)
    
    # Call the function to get updated arrays for policy3 without ordering
    t_LC_array, costs_rep_array, costs_delay_array, costs_stock_array = calculate_probabilities_for_pdm_policy_3_Without_ordering(estimator, pdm_df,scaler,train_df, params, cols_normalize_train, sequence_cols, C_p, C_c, L,DT, C_unav, C_inv, device)
    # Calculate M_hat using the updated arrays
    M_hat6 = calculate_decision_metric_without_ordering(costs_rep_array,  t_LC_array, C_p, pdm_df)
    M_hat_Policy3_Without_ordering.append(M_hat6)
    
    
    

current_cycle:  0
current_cycle:  0 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  10
current_cycle:  10 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  20
current_cycle:  20 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  30
current_cycle:  30 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  40
current_cycle:  40 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  50
current_cycle:  50 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  60
current_cycle:  60 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  70
current_cycle:  70 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  80
current_cycle:  80 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  90
current_cycle:  90 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  100
current_cycle:  100 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  110
current_cycle:  110 || DT:  10 ||T_R_k_optima

current_cycle:  10 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  20
current_cycle:  20 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  30
current_cycle:  30 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  40
current_cycle:  40 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  50
current_cycle:  50 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  60
current_cycle:  60 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  70
current_cycle:  70 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  80
current_cycle:  80 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  90
current_cycle:  90 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  100
current_cycle:  100 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  110
current_cycle:  110 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  120
current_cycle:  120 || DT:  10 ||T_R_k_optimal:  19.99999335

current_cycle:  70 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  80
current_cycle:  80 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  90
current_cycle:  90 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  100
current_cycle:  100 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  110
current_cycle:  110 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  120
current_cycle:  120 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  130
current_cycle:  130 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  140
current_cycle:  140 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  150
current_cycle:  150 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  160
current_cycle:  160 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  170
current_cycle:  170 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  180
current_cycle:  180 || DT:  10 ||T_R_k_optimal: 

current_cycle:  170 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  180
current_cycle:  180 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  190
current_cycle:  190 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  200
current_cycle:  200 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  210
current_cycle:  210 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  220
current_cycle:  220 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  230
current_cycle:  230 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  240
current_cycle:  240 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  250
current_cycle:  250 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  260
current_cycle:  260 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  270
current_cycle:  270 || DT:  10 ||T_R_k_optimal:  19.99999335625205
current_cycle:  280
current_cycle:  280 || DT:  10 ||T_R_k_opti

KeyboardInterrupt: 

In [ ]:
# Calculate C_p/C_c ratios
C_p_C_c_ratios = C_p_values / C_c
# Plotting the results
plt.figure(figsize=(12, 8))
plt.plot(C_p_C_c_ratios, M_hat_Policy1_With_ordering, marker='o', label='Policy 1 With Ordering')
plt.plot(C_p_C_c_ratios, M_hat_Policy1_Without_ordering, marker='o', label='Policy 1 Without Ordering')
plt.plot(C_p_C_c_ratios, M_hat_Policy2_With_ordering, marker='o', label='Policy 2 With Ordering')
plt.plot(C_p_C_c_ratios, M_hat_Policy2_Without_ordering, marker='o', label='Policy 2 Without Ordering')
plt.plot(C_p_C_c_ratios, M_hat_Policy3_With_ordering, marker='o', label='Policy 3 With Ordering')
plt.plot(C_p_C_c_ratios, M_hat_Policy3_Without_ordering, marker='o', label='Policy 3 Without Ordering')

plt.title('Decision Metric M_hat vs C_p/C_c Ratio')
plt.xlabel('C_p/C_c Ratio')
plt.ylabel('Decision Metric M_hat')
plt.legend()
plt.yscale("log")
plt.grid()
plt.xticks(C_p_C_c_ratios)  # Set x-ticks to the calculated ratios
plt.axhline(0, color='black', lw=0.5, ls='--')  # Add a horizontal line at y=0
plt.show()

In [84]:
"""
from sklearn import preprocessing

counter = 0
for id in test_df['id'].unique():
    print('ID:', id)
    preventive_replacement = False
    order = False

    for current_cycle in range(test_df[test_df['id'] == id].shape[0] - params['seq_length'] + 1):

        if current_cycle in array_decisions:
            min_max_scaler = preprocessing.MinMaxScaler()
            norm_test_df = pd.DataFrame(min_max_scaler.fit_transform(test_df[test_df['id'] == id][cols_normalize_train][:params['seq_length'] + current_cycle]),
                                        columns=cols_normalize_train,
                                        index=test_df[test_df['id'] == id][:params['seq_length'] + current_cycle].index)

            join_df = test_df[test_df['id'] == id][:params['seq_length'] + current_cycle][test_df[test_df['id'] == id][:params['seq_length'] + current_cycle].columns.difference(cols_normalize_train)].join(norm_test_df)
            test_df_eval_online = join_df.reindex(columns=test_df[test_df['id'] == id][current_cycle:params['seq_length'] + current_cycle].columns)

            seq_array_test_k = test_df_eval_online[sequence_cols].values[current_cycle:params['seq_length'] + current_cycle]
            seq_array_test_k = np.asarray(seq_array_test_k).astype(np.float32).reshape(1, params['seq_length'], nb_features)

            # Convert to tensor
            seq_tensor = torch.tensor(seq_array_test_k, dtype=torch.float32).to(device)

            # Predict
            outputs = estimator(seq_tensor).squeeze()
            #print('outputs: ',outputs )
            probabilities = torch.softmax(outputs, dim=1).cpu().detach().numpy() #shape: [batch_size, sequence_length,num_classes]
            #prob_RUL_smaller_DT = probabilities[-1,2] # class2 probability for last time steps
            #prob_RUL_smaller_w1 = probabilities[-1,1] + probabilities[-1,2] # (class1+class2) for last time steps
            
            # Assuming 'probabilities' is your (1, 50, 3) tensor
            prob_RUL_smaller_w1 = probabilities[:, 1] + probabilities[:, 2]
            prob_RUL_smaller_DT = probabilities[:, 2]

            
            #print('Collective probabilities: ', probabilities)
            #print('shape of Collective probabilities: ', probabilities.shape)
            print('prob_RUL_smaller_w1:', prob_RUL_smaller_w1)
            print('prob_RUL_smaller_DT:', prob_RUL_smaller_DT)

            # Evaluate decision heuristics
            if not order:#
                # Order component
                if C_p <= ( * C_c):
                    print('prob_RUL_smaller_w1:', prob_RUL_smaller_w1)
                    print('prob_RUL_smaller_DT:', prob_RUL_smaller_DT)
                    t_order_array[counter] = params['seq_length'] + current_cycle
                    order = True
                    print('component ordering at cycle:', t_order_array[counter])
            
            # Perform preventive replacement
            if C_p <= (prob_RUL_smaller_DT * C_c):
                print('prob_RUL_smaller_DT:', prob_RUL_smaller_DT)

                t_LC_array[counter] = params['seq_length'] + current_cycle
                costs_rep_array[counter] = C_p
                print('preventive replacement informed at cycle:', t_LC_array[counter])

                preventive_replacement = True
                costs_delay_array[counter] = max(t_order_array[counter] + L - t_LC_array[counter], 0) * C_unav
                costs_stock_array[counter] = max(t_LC_array[counter] - (t_order_array[counter] + L), 0) * C_inv
                break

    if not preventive_replacement:
        t_LC_array[counter] = test_df[test_df['id'] == id]['cycle'].iloc[-1]
        print('Component failure at t:', t_LC_array[counter])
        costs_rep_array[counter] = C_c

        if not order:
            costs_delay_array[counter] = L * C_unav
        else:
            costs_delay_array[counter] = max(t_order_array[counter] + L - t_LC_array[counter], 0) * C_unav
            costs_stock_array[counter] = max(t_LC_array[counter] - (t_order_array[counter] + L), 0) * C_inv

    print('True failure:', test_df[test_df['id'] == id]['cycle'].iloc[-1])
    print('-----------------------------------------')
    counter += 1


"""

"\nfrom sklearn import preprocessing\n\ncounter = 0\nfor id in test_df['id'].unique():\n    print('ID:', id)\n    preventive_replacement = False\n    order = False\n\n    for current_cycle in range(test_df[test_df['id'] == id].shape[0] - params['seq_length'] + 1):\n\n        if current_cycle in array_decisions:\n            min_max_scaler = preprocessing.MinMaxScaler()\n            norm_test_df = pd.DataFrame(min_max_scaler.fit_transform(test_df[test_df['id'] == id][cols_normalize_train][:params['seq_length'] + current_cycle]),\n                                        columns=cols_normalize_train,\n                                        index=test_df[test_df['id'] == id][:params['seq_length'] + current_cycle].index)\n\n            join_df = test_df[test_df['id'] == id][:params['seq_length'] + current_cycle][test_df[test_df['id'] == id][:params['seq_length'] + current_cycle].columns.difference(cols_normalize_train)].join(norm_test_df)\n            test_df_eval_online = join_df.reindex(

In [85]:
costs_tot

NameError: name 'costs_tot' is not defined

In [ ]:
t_LC_array

In [ ]:
t_order_array

### This code calculates the expected cost per unit time using the LSTM model. It computes the mean of the total costs divided by the mean of the time to component failure (t_LC_array). This metric gives an estimate of the average cost incurred per unit time in the system, considering both maintenance and operational costs.

In [ ]:
expected_cost_LSTM = np.mean(costs_tot) / np.mean(t_LC_array)
expected_cost_LSTM

This code segment calculates the expected cost per unit time assuming perfect prognostics.
`1. Perfect Prognostics Calculation:`
> * It initializes an array `t_LC_perfect_array` to store the time of component failure for each unit in the validation dataset. This is calculated by dividing the last observed cycle number by the decision interval DT and then flooring the result to get the last decision cycle before failure.
> * The loop iterates over each unique ID in the validation dataset, calculates the time of component failure for each unit, and stores it in
`t_LC_perfect_array`.<br>
> * `math.floor()` is used to round down the result to the nearest multiple of `DT`.
> * Finally, the loop increments the counter for each unit.<br>

`2. Cost Calculation:`
> * `costs_perfect_array` is initialized with a value of `C_p`, representing the cost of preventive replacements. In a perfect scenario, only preventive replacements are made.
> * This array holds the same cost value for each unit in the validation dataset.

`3. Expected Cost Calculation:`
> * `expected_cost_perfect` is calculated by taking the mean of `costs_perfect_array` and dividing it by the mean of `t_LC_perfect_array`.
> * This calculation provides an estimate of the average cost per unit time assuming perfect prognostics, where components are replaced preventively at regular intervals.









In [ ]:
# Perfect prognostics
import math
t_LC_perfect_array  = np.zeros(10)
DT=10
counter=0
for id in test_df['id'].unique():
    t_LC_perfect_array[counter] = math.floor(test_df[test_df['id']==id]['cycle'].iloc[-1] /DT) * DT
    counter+=1

costs_perfect_array = np.ones(10)*C_p # a perfect policy will only lead to preventive replacements

expected_cost_perfect = np.mean(costs_perfect_array)/np.mean(t_LC_perfect_array)
expected_cost_perfect


In [ ]:
t_LC_perfect_array

In [ ]:
# evaluation of the metric defined in the paper
M = (expected_cost_LSTM - expected_cost_perfect) / expected_cost_perfect
M # it obtains a very small value

In [ ]:
M*100